In [1]:
from cerebralcortex import Kernel
import json
# from IPython.display import display
# import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# import plotly.graph_objs as go
# import plotly.figure_factory as ff
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, ModuleMetadata
from cerebralcortex.core.util.spark_helper import get_or_create_sc
import ipywidgets as widgets


init_notebook_mode(connected=True)
pd.set_option('display.max_rows',5)

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
CC = Kernel("/home/jupyter/cc3_conf/", study_name='mperf')

In [4]:
CC.search_stream('gyro')

['/cc3/study=mperf/gyroscope--org.md2k.motionsense--motion_sense_hrv--left_wrist',
 '/cc3/study=mperf/gyroscope--org.md2k.motionsense--motion_sense_hrv--right_wrist',
 '/cc3/study=mperf/gyroscope--org.md2k.phonesensor--phone']

In [15]:
accelerometer_right_wrist_stream_name = 'accelerometer--org.md2k.motionsense--motion_sense_hrv--right_wrist'
gyroscope_right_wrist_stream_name = 'gyroscope--org.md2k.motionsense--motion_sense_hrv--right_wrist'

In [13]:
user_id = CC.list_users()[0]['user_id']
user_name = CC.get_user_name(user_id)

In [16]:
accel_ds = CC.get_stream(accelerometer_right_wrist_stream_name).filter_user(user_id).filter("accelerometer_x >= -4 AND accelerometer_x <= 4 AND accelerometer_y >= -4 AND accelerometer_y <= 4 AND accelerometer_z >= -4 AND accelerometer_z <= 4")

In [17]:
gyro_ds = CC.get_stream(gyroscope_right_wrist_stream_name).filter_user(user_id).filter("gyroscope_x >= -250 AND gyroscope_x <= 250 AND gyroscope_y >= -250 AND gyroscope_y <= 250 AND gyroscope_z >= -250 AND gyroscope_z <= 250")

In [18]:
accel_w = accel_ds.window(windowDuration=20)
gyro_w = gyro_ds.window(windowDuration=20)

In [19]:
gyro_w = gyro_w.drop("version", "user")

In [20]:
common_ds = accel_w.join(gyro_w, on="window")

In [21]:
common_ds = common_ds.withColumn('ts', F.udf(lambda w: w[0], TimestampType())('window'))

In [22]:
common_ds = common_ds.withColumn('timestamp', F.udf(lambda w: w[0], TimestampType())('window')).drop('window')

In [108]:
def is_correct_accel_data(data):
#     data = np.array([accel_x, accel_y, accel_z]).T
    mg = np.mean([np.linalg.norm(x) for x in data])
    if mg < 0.8:
        return 0
    return 1
udf_is_correct_accel_data = F.udf(is_correct_accel_data, IntegerType())

def correct_accel(data, is_correct):
    if is_correct == 1:
        return data
    data = np.array(data)*2
    return data.tolist()
udf_correct_accel = F.udf(correct_accel, ArrayType(DoubleType()))


In [145]:
schema = StructType([StructField("timestamp", TimestampType()), StructField("accel", ArrayType(ArrayType(DoubleType()))), StructField("gyro", ArrayType(ArrayType(DoubleType())))])
# schema = StructType([StructField("timestamp", TimestampType()), StructField("data", IntegerType())])
@F.pandas_udf(schema, F.PandasUDFType.GROUPED_MAP)
def get_data_udf(user_data):
    if len(user_data.index) < 900:
        return pd.DataFrame([], columns=['timestamp', 'accel', 'gyro'])
    is_movement = []
    correct_data = []
    for index, row in user_data.iterrows():
        al = len(row['accelerometer_x'])
        gl = len(row['gyroscope_x'])
        if al<300:  # 25 * 20 * 60% (frequency * window_size * threshold%)
            is_movement.append(0)
            correct_data.append(1)
            continue
    
        accel_data = np.array([*row['accelerometer_x'], *row['accelerometer_y'], *row['accelerometer_z']])
        
        accel_data = accel_data.reshape(-1, al)
        accel_data = accel_data.T
        if not is_correct_accel_data(accel_data):
            ax = 2*np.array(row['accelerometer_x']).tolist()
            ay = 2*np.array(row['accelerometer_y']).tolist()
            az = 2*np.array(row['accelerometer_z']).tolist()
            accel_data = np.array([*ax, *ay, *az])
            accel_data = accel_data.reshape(-1, al)
            accel_data = accel_data.T
            correct_data.append(0)
        else:
            correct_data.append(1)
        mg = [np.linalg.norm(x) for x in accel_data]
        sd= np.std(mg)
        if sd >= 0.21:
            is_movement.append(1)
        else:
            is_movement.append(0)
    user_data['is_movement'] = is_movement
    user_data['correct'] = correct_data
    data_movement = user_data[user_data['is_movement']==1]
    if len(data_movement.index) < 900:
        return pd.DataFrame([], columns=['timestamp', 'accel', 'gyro'])
    leap = int(np.floor(len(data_movement.index) / 900))
    ts = []
    adt = []
    gdt = []
    for i in range(0, len(data_movement.index), leap):
        row = data_movement.iloc[i]
        al = len(row['accelerometer_x'])
        gl = len(row['gyroscope_x'])
        ts.append(row['timestamp'])
        if row['correct']==1:
            accel_data = np.array([*row['accelerometer_x'], *row['accelerometer_y'], *row['accelerometer_z']])
        else:
            ax = 2*np.array(row['accelerometer_x']).tolist()
            ay = 2*np.array(row['accelerometer_y']).tolist()
            az = 2*np.array(row['accelerometer_z']).tolist()
            accel_data = np.array([*ax, *ay, *az])
                                   
        gyro_data = np.array([*row['gyroscope_x'], *row['gyroscope_y'], *row['gyroscope_z']])
        accel_data = accel_data.reshape(-1, al)
        gyro_data = gyro_data.reshape(-1, gl)
        accel_data = accel_data.T
        gyro_data = gyro_data.T
        
        adt.append(accel_data.tolist())
        gdt.append(gyro_data.tolist())
        if len(ts)>=900:
            break
    return pd.DataFrame(data={'timestamp': ts, 'accel': adt, 'gyro': gdt})

In [146]:
windowDuration = "1 day"
groupbycols = ["user", "version"]

win = F.window("timestamp", windowDuration=windowDuration)

groupbycols.append(win)

data = common_ds._data.groupBy(groupbycols).apply(get_data_udf)

In [147]:
# temp = common_ds.compute(get_data_udf, windowDuration=60*60*24)

In [148]:
pdf = data.toPandas()

In [153]:
pdf.iloc[0]

timestamp                                  2017-11-21 08:13:20
accel        [[-0.234130859375, 0.093017578125, 0.997924804...
gyro         [[-1.3427734375, 0.579833984375, 2.34985351562...
Name: 0, dtype: object

In [30]:
temp = common_ds._data
temp.filter(temp.timestamp==datetime(2017,12,15,5,52,20)).show(truncate=False)

+------------------------------------+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
pdf['data'].iloc[0]

[[-0.07421875,
  -0.0731201171875,
  -0.0740966796875,
  -0.073486328125,
  -0.07421875,
  -0.0723876953125,
  -0.0731201171875,
  -0.073486328125,
  -0.072998046875,
  -0.073974609375,
  -0.072998046875,
  -0.0732421875,
  -0.0732421875,
  -0.073486328125,
  -0.07275390625,
  -0.0714111328125,
  -0.0736083984375,
  -0.0728759765625,
  -0.07275390625,
  -0.0728759765625,
  -0.07177734375,
  -0.0721435546875,
  -0.072998046875,
  -0.0723876953125,
  -0.072509765625,
  -0.072265625,
  -0.0728759765625,
  -0.072998046875,
  -0.0723876953125,
  -0.0731201171875,
  -0.0731201171875,
  -0.072509765625,
  -0.0718994140625,
  -0.072998046875,
  -0.0721435546875,
  -0.0726318359375,
  -0.0723876953125,
  -0.072509765625,
  -0.0718994140625,
  -0.0728759765625,
  -0.0736083984375,
  -0.0733642578125,
  -0.074462890625,
  -0.07275390625,
  -0.0733642578125,
  -0.072998046875,
  -0.0733642578125,
  -0.0733642578125,
  -0.072998046875,
  -0.0728759765625,
  -0.0733642578125,
  -0.0726318359375,
  -

In [80]:
pdf.to_csv("data/"+user_id+".csv", index=False)

In [85]:
common_ds.show(5, truncate=False)

+------------------------------------+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [88]:
pdf.to_csv('data/'+user_id+".csv.gz", compression='gzip')

In [82]:
pd.read_csv("data/"+user_id+".csv")

,timestamp,data
0,2017-12-15 05:58:40,"[[-0.0728759765625, -0.072265625, -0.072631835..."
1,2017-12-15 15:11:40,"[[0.3101806640625, 0.325439453125, 0.315429687..."
...,...,...
10798,2017-12-05 14:43:20,"[[-0.563232421875, -0.57177734375, -0.61608886..."
10799,2017-12-05 14:56:00,"[[0.36474609375, 0.3912353515625, 0.3955078125..."


In [130]:
fake = [[datetime.now()]*1000, [[1]*5]*900+[[0.7]*5]*100, [[2]*5]*900+[[0.7]*5]*100, [[3]*5]*900+[[0.7]*5]*100, [[5]*5]*900+[[10]*5]*100,[[6]*5]*900+[[10]*5]*100, [[7]*5]*900+[[10]*5]*100]

In [131]:
test_df = pd.DataFrame(data={'timestamp': fake[0], 'accelerometer_x': fake[1], 'accelerometer_y': fake[2], 'accelerometer_z': fake[3], 'gyroscope_x': fake[4], 'gyroscope_y': fake[5], 'gyroscope_z': fake[6]})

In [132]:
test_df

,timestamp,accelerometer_x,accelerometer_y,accelerometer_z,gyroscope_x,gyroscope_y,gyroscope_z
0,2019-12-10 04:22:07.164561,"[1, 1, 1, 1, 1]","[2, 2, 2, 2, 2]","[3, 3, 3, 3, 3]","[5, 5, 5, 5, 5]","[6, 6, 6, 6, 6]","[7, 7, 7, 7, 7]"
1,2019-12-10 04:22:07.164561,"[1, 1, 1, 1, 1]","[2, 2, 2, 2, 2]","[3, 3, 3, 3, 3]","[5, 5, 5, 5, 5]","[6, 6, 6, 6, 6]","[7, 7, 7, 7, 7]"
...,...,...,...,...,...,...,...
998,2019-12-10 04:22:07.164561,"[0.7, 0.7, 0.7, 0.7, 0.7]","[0.7, 0.7, 0.7, 0.7, 0.7]","[0.7, 0.7, 0.7, 0.7, 0.7]","[10, 10, 10, 10, 10]","[10, 10, 10, 10, 10]","[10, 10, 10, 10, 10]"
999,2019-12-10 04:22:07.164561,"[0.7, 0.7, 0.7, 0.7, 0.7]","[0.7, 0.7, 0.7, 0.7, 0.7]","[0.7, 0.7, 0.7, 0.7, 0.7]","[10, 10, 10, 10, 10]","[10, 10, 10, 10, 10]","[10, 10, 10, 10, 10]"


In [133]:
# schema = StructType([StructField("timestamp", TimestampType()), StructField("accel", ArrayType(ArrayType(DoubleType()))), StructField("gyro", ArrayType(ArrayType(DoubleType())))])
# schema = StructType([StructField("timestamp", TimestampType()), StructField("data", IntegerType())])
# @F.pandas_udf(schema, F.PandasUDFType.GROUPED_MAP)
def get_data_test(user_data):
    is_movement = []
    correct_data = []
    for index, row in user_data.iterrows():
        al = len(row['accelerometer_x'])
        gl = len(row['gyroscope_x'])
#         if al<300:  # 25 * 20 * 60% (frequency * window_size * threshold%)
#             is_movement.append(0)
#             correct_data.append(1)
#             continue
    
        accel_data = np.array([*row['accelerometer_x'], *row['accelerometer_y'], *row['accelerometer_z']])
        
        accel_data = accel_data.reshape(-1, al)
        accel_data = accel_data.T
        if not is_correct_accel_data(accel_data):
            ax = 2*np.array(row['accelerometer_x']).tolist()
            ay = 2*np.array(row['accelerometer_y']).tolist()
            az = 2*np.array(row['accelerometer_z']).tolist()
            accel_data = np.array([*ax, *ay, *az])
            accel_data = accel_data.reshape(-1, al)
            accel_data = accel_data.T
            correct_data.append(0)
        else:
            correct_data.append(1)
        mg = [np.linalg.norm(x) for x in accel_data]
        sd= np.std(mg)
        if sd >= 0:
            is_movement.append(1)
        else:
            is_movement.append(0)
    user_data['is_movement'] = is_movement
    user_data['correct'] = correct_data
    data_movement = user_data[user_data['is_movement']==1]
#     if len(data_movement.index) < 900:
#         return pd.DataFrame([], columns=['timestamp', 'data'])
#     leap = int(np.floor(len(data_movement.index) / 900))
    leap = 1
    ts = []
    adt = []
    gdt = []
    for i in range(0, len(data_movement.index), leap):
        row = data_movement.iloc[i]
        al = len(row['accelerometer_x'])
        gl = len(row['gyroscope_x'])
        ts.append(row['timestamp'])
        if row['correct']==1:
            accel_data = np.array([*row['accelerometer_x'], *row['accelerometer_y'], *row['accelerometer_z']])
        else:
            ax = 2*np.array(row['accelerometer_x']).tolist()
            ay = 2*np.array(row['accelerometer_y']).tolist()
            az = 2*np.array(row['accelerometer_z']).tolist()
            accel_data = np.array([*ax, *ay, *az])
                                   
        gyro_data = np.array([*row['gyroscope_x'], *row['gyroscope_y'], *row['gyroscope_z']])
        accel_data = accel_data.reshape(-1, al)
        gyro_data = gyro_data.reshape(-1, gl)
        accel_data = accel_data.T
        gyro_data = gyro_data.T
        
        adt.append(accel_data.tolist())
        gdt.append(gyro_data.tolist())
        if len(ts)>=900:
            break
    return pd.DataFrame(data={'timestamp': ts, 'accel': adt, 'gyro': gdt})

In [134]:
get_data_test(test_df)

,timestamp,accel,gyro
0,2019-12-10 04:22:07.164561,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [...","[[5, 6, 7], [5, 6, 7], [5, 6, 7], [5, 6, 7], [..."
1,2019-12-10 04:22:07.164561,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [...","[[5, 6, 7], [5, 6, 7], [5, 6, 7], [5, 6, 7], [..."
...,...,...,...
898,2019-12-10 04:22:07.164561,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [...","[[5, 6, 7], [5, 6, 7], [5, 6, 7], [5, 6, 7], [..."
899,2019-12-10 04:22:07.164561,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [...","[[5, 6, 7], [5, 6, 7], [5, 6, 7], [5, 6, 7], [..."


In [ ]:
fake[4]

In [ ]:
1

In [41]:
ax = [-0.0740966796875, -0.07373046875, -0.0733642578125, -0.07421875, -0.07373046875, -0.0738525390625, -0.0743408203125, -0.07421875, -0.0731201171875, -0.0731201171875, -0.0743408203125, -0.073974609375, -0.07373046875, -0.0732421875, -0.0745849609375, -0.0743408203125, -0.0740966796875, -0.073974609375, -0.07373046875, -0.07421875, -0.0726318359375, -0.074951171875, -0.0740966796875, -0.0751953125, -0.073974609375, -0.073974609375, -0.07421875, -0.073974609375, -0.0738525390625, -0.0731201171875, -0.074462890625, -0.0733642578125, -0.07373046875, -0.0738525390625, -0.0733642578125, -0.07373046875, -0.0740966796875, -0.073974609375, -0.0745849609375, -0.0740966796875, -0.0743408203125, -0.0738525390625, -0.074462890625, -0.0736083984375, -0.0736083984375, -0.0738525390625, -0.07568359375, -0.0736083984375, -0.0728759765625, -0.0743408203125, -0.074462890625, -0.07470703125, -0.07373046875, -0.0731201171875, -0.074462890625, -0.0738525390625, -0.07373046875, -0.0736083984375, -0.074462890625, -0.0728759765625, -0.0748291015625, -0.07373046875, -0.073486328125, -0.073974609375, -0.0748291015625, -0.0743408203125, -0.0750732421875, -0.073486328125, -0.0733642578125, -0.0738525390625, -0.072509765625, -0.073486328125, -0.0732421875, -0.0745849609375, -0.0733642578125, -0.072509765625, -0.0745849609375, -0.07470703125, -0.0736083984375, -0.074462890625, -0.07373046875, -0.0738525390625, -0.073486328125, -0.0748291015625, -0.07373046875, -0.0732421875, -0.0743408203125, -0.074462890625, -0.075439453125, -0.0743408203125, -0.073974609375, -0.0748291015625, -0.073486328125, -0.07421875, -0.07421875, -0.07421875, -0.074951171875, -0.074462890625, -0.073974609375, -0.0745849609375, -0.073486328125, -0.07421875, -0.07421875, -0.073974609375, -0.0738525390625, -0.07373046875, -0.074462890625, -0.073486328125, -0.0732421875, -0.07275390625, -0.0738525390625, -0.0740966796875, -0.0740966796875, -0.0738525390625, -0.0745849609375, -0.0740966796875, -0.0736083984375, -0.07373046875, -0.074951171875, -0.07373046875, -0.0736083984375, -0.0745849609375, -0.0743408203125, -0.0738525390625, -0.0743408203125, -0.0740966796875, -0.073486328125, -0.07373046875, -0.0733642578125, -0.0738525390625, -0.0743408203125, -0.0733642578125, -0.073486328125, -0.074951171875, -0.0733642578125, -0.074462890625, -0.073486328125, -0.073486328125, -0.074462890625, -0.0736083984375, -0.07421875, -0.073974609375, -0.074462890625, -0.074462890625, -0.073974609375, -0.0733642578125, -0.0748291015625, -0.07421875, -0.074951171875, -0.07275390625, -0.0740966796875, -0.0743408203125, -0.07470703125, -0.0740966796875, -0.074462890625, -0.0740966796875, -0.073974609375, -0.0733642578125, -0.07470703125, -0.074462890625, -0.0738525390625, -0.0740966796875, -0.0733642578125, -0.07421875, -0.0732421875, -0.074462890625, -0.0748291015625, -0.07470703125, -0.0745849609375, -0.0736083984375, -0.07421875, -0.07421875, -0.074462890625, -0.07470703125, -0.0738525390625, -0.0751953125, -0.07373046875, -0.07421875, -0.0745849609375, -0.0736083984375, -0.0748291015625, -0.073974609375, -0.074462890625, -0.0732421875, -0.0738525390625, -0.0740966796875, -0.0733642578125, -0.07373046875, -0.0740966796875, -0.07373046875, -0.0736083984375, -0.0745849609375, -0.074462890625, -0.0736083984375, -0.07421875, -0.0740966796875, -0.0743408203125, -0.07373046875, -0.0736083984375, -0.073974609375, -0.0740966796875, -0.07373046875, -0.0738525390625, -0.0740966796875, -0.0731201171875, -0.0736083984375, -0.073974609375, -0.0745849609375, -0.0738525390625, -0.074951171875, -0.07373046875, -0.0738525390625, -0.0736083984375, -0.0748291015625, -0.073974609375, -0.07421875, -0.073486328125, -0.0745849609375, -0.074462890625, -0.073486328125, -0.07373046875, -0.073486328125, -0.0738525390625, -0.0736083984375, -0.0732421875, -0.0736083984375, -0.0751953125, -0.0743408203125, -0.0740966796875, -0.0736083984375, -0.07373046875, -0.073974609375, -0.07470703125, -0.0753173828125, -0.073974609375, -0.0740966796875, -0.0738525390625, -0.0740966796875, -0.0743408203125, -0.074951171875, -0.0745849609375, -0.074462890625, -0.0728759765625, -0.07421875, -0.0745849609375, -0.072998046875, -0.0738525390625, -0.073974609375, -0.07421875, -0.07373046875, -0.0743408203125, -0.074462890625, -0.073974609375, -0.0740966796875, -0.0745849609375, -0.07421875, -0.0728759765625, -0.073974609375, -0.073974609375, -0.0748291015625, -0.074462890625, -0.074951171875, -0.07470703125, -0.0748291015625, -0.074462890625, -0.074462890625, -0.0732421875, -0.0743408203125, -0.0738525390625, -0.073974609375, -0.0750732421875, -0.0750732421875, -0.0728759765625, -0.0738525390625, -0.0738525390625, -0.0745849609375, -0.0743408203125, -0.07470703125, -0.07470703125, -0.073974609375, -0.0750732421875, -0.073974609375, -0.074951171875, -0.072509765625, -0.0743408203125, -0.0740966796875, -0.07421875, -0.0738525390625, -0.0736083984375, -0.073486328125, -0.07373046875, -0.074462890625, -0.07421875, -0.07421875, -0.073486328125, -0.074462890625, -0.074462890625, -0.073486328125, -0.073974609375, -0.072998046875, -0.0748291015625, -0.0733642578125, -0.0736083984375, -0.0732421875, -0.07373046875, -0.07373046875, -0.0736083984375, -0.074462890625, -0.0736083984375, -0.0743408203125, -0.073974609375, -0.0740966796875, -0.07373046875, -0.07373046875, -0.0733642578125, -0.07421875, -0.073974609375, -0.0743408203125, -0.07421875, -0.0736083984375, -0.07421875, -0.0736083984375, -0.0743408203125, -0.07421875, -0.073974609375, -0.074951171875, -0.07373046875, -0.0740966796875, -0.0738525390625, -0.0732421875, -0.0736083984375, -0.0743408203125, -0.0736083984375, -0.073974609375, -0.074462890625, -0.0748291015625, -0.073974609375, -0.07470703125, -0.0738525390625, -0.074951171875, -0.074462890625, -0.0731201171875, -0.0740966796875, -0.0736083984375, -0.07373046875, -0.073974609375, -0.0731201171875, -0.0728759765625, -0.0736083984375, -0.07421875, -0.07373046875, -0.073486328125, -0.074462890625, -0.0731201171875, -0.0740966796875, -0.0743408203125, -0.074462890625, -0.073974609375, -0.0743408203125, -0.073974609375, -0.07373046875, -0.072998046875, -0.07421875, -0.0733642578125, -0.07373046875, -0.0743408203125, -0.0736083984375, -0.0732421875, -0.0745849609375, -0.073486328125, -0.0733642578125, -0.0733642578125, -0.0745849609375, -0.074462890625, -0.0733642578125, -0.07373046875, -0.0738525390625, -0.07373046875, -0.073486328125, -0.0733642578125, -0.07373046875, -0.07470703125, -0.07421875, -0.072509765625, -0.073486328125, -0.073974609375, -0.0740966796875, -0.0738525390625, -0.0740966796875, -0.0740966796875, -0.0740966796875, -0.073486328125, -0.07421875, -0.0736083984375, -0.07421875, -0.07421875, -0.0743408203125, -0.073974609375, -0.0738525390625, -0.0733642578125, -0.0743408203125, -0.0740966796875, -0.0740966796875, -0.0743408203125, -0.07470703125, -0.073486328125, -0.07421875, -0.0733642578125, -0.0731201171875, -0.0738525390625, -0.07470703125, -0.0743408203125, -0.0738525390625, -0.0738525390625, -0.07421875, -0.0736083984375, -0.074462890625, -0.073974609375, -0.07421875, -0.0743408203125, -0.0740966796875, -0.0745849609375, -0.0745849609375, -0.074462890625, -0.0731201171875, -0.07373046875, -0.0745849609375, -0.07421875, -0.0732421875, -0.0736083984375, -0.0732421875, -0.073974609375, -0.0738525390625, -0.073974609375, -0.0732421875, -0.07373046875, -0.0743408203125, -0.0748291015625, -0.07470703125, -0.073486328125, -0.0731201171875, -0.0733642578125, -0.07373046875, -0.0743408203125, -0.0736083984375, -0.0732421875, -0.07373046875, -0.0732421875, -0.07373046875, -0.0738525390625, -0.0731201171875, -0.0738525390625, -0.073486328125, -0.0738525390625, -0.073974609375, -0.0731201171875, -0.072998046875, -0.0740966796875, -0.0733642578125, -0.0738525390625, -0.0736083984375, -0.0740966796875, -0.07373046875, -0.073974609375, -0.0743408203125, -0.0740966796875, -0.072998046875, -0.07470703125, -0.0731201171875, -0.07373046875, -0.07373046875, -0.0736083984375, -0.0743408203125, -0.074951171875, -0.0738525390625, -0.07421875, -0.074462890625, -0.0740966796875, -0.0745849609375, -0.07470703125, -0.074462890625, -0.07421875, -0.0740966796875, -0.073974609375, -0.0743408203125, -0.073974609375, -0.0736083984375, -0.0740966796875, -0.073486328125, -0.0738525390625, -0.073974609375, -0.0738525390625, -0.07421875, -0.0733642578125, -0.074462890625, -0.074462890625, -0.0748291015625, -0.0743408203125, -0.0745849609375, -0.0743408203125]

In [33]:
ttt2 = [-0.07421875,
  -0.0731201171875,
  -0.0740966796875,
  -0.073486328125,
  -0.07421875,
  -0.0723876953125,
  -0.0731201171875,
  -0.073486328125,
  -0.072998046875,
  -0.073974609375,
  -0.072998046875,
  -0.0732421875,
  -0.0732421875,
  -0.073486328125,
  -0.07275390625,
  -0.0714111328125,
  -0.0736083984375,
  -0.0728759765625,
  -0.07275390625,
  -0.0728759765625,
  -0.07177734375,
  -0.0721435546875,
  -0.072998046875,
  -0.0723876953125,
  -0.072509765625,
  -0.072265625,
  -0.0728759765625,
  -0.072998046875,
  -0.0723876953125,
  -0.0731201171875,
  -0.0731201171875,
  -0.072509765625,
  -0.0718994140625,
  -0.072998046875,
  -0.0721435546875,
  -0.0726318359375,
  -0.0723876953125,
  -0.072509765625,
  -0.0718994140625,
  -0.0728759765625,
  -0.0736083984375,
  -0.0733642578125,
  -0.074462890625,
  -0.07275390625,
  -0.0733642578125,
  -0.072998046875,
  -0.0733642578125,
  -0.0733642578125,
  -0.072998046875,
  -0.0728759765625,
  -0.0733642578125,
  -0.0726318359375,
  -0.07275390625,
  -0.07177734375,
  -0.0716552734375,
  -0.07275390625,
  -0.0723876953125,
  -0.0726318359375,
  -0.0721435546875,
  -0.072509765625,
  -0.07177734375,
  -0.072265625,
  -0.072265625,
  -0.0728759765625,
  -0.07275390625,
  -0.07373046875,
  -0.072998046875,
  -0.0732421875,
  -0.072998046875,
  -0.073486328125,
  -0.0732421875,
  -0.0733642578125,
  -0.0732421875,
  -0.0731201171875,
  -0.072998046875,
  -0.07275390625,
  -0.07275390625,
  -0.07275390625,
  -0.0743408203125,
  -0.0733642578125,
  -0.07275390625,
  -0.0728759765625,
  -0.0733642578125,
  -0.0723876953125,
  -0.07275390625,
  -0.072021484375,
  -0.07275390625,
  -0.07373046875,
  -0.072998046875,
  -0.0728759765625,
  -0.0728759765625,
  -0.0731201171875,
  -0.0728759765625,
  -0.0723876953125,
  -0.07275390625,
  -0.0740966796875,
  -0.07275390625,
  -0.072509765625,
  -0.072509765625,
  -0.072998046875,
  -0.0723876953125,
  -0.0721435546875,
  -0.07275390625,
  -0.0736083984375,
  -0.0733642578125,
  -0.0726318359375,
  -0.072509765625,
  -0.0723876953125,
  -0.0728759765625,
  -0.0726318359375,
  -0.0731201171875,
  -0.0731201171875,
  -0.0726318359375,
  -0.073486328125,
  -0.0723876953125,
  -0.0731201171875,
  -0.0716552734375,
  -0.072021484375,
  -0.0732421875,
  -0.0732421875,
  -0.07275390625,
  -0.0731201171875,
  -0.0714111328125,
  -0.0726318359375,
  -0.0731201171875,
  -0.07373046875,
  -0.073486328125,
  -0.0721435546875,
  -0.072265625,
  -0.0728759765625,
  -0.0723876953125,
  -0.072509765625,
  -0.0726318359375,
  -0.072998046875,
  -0.0721435546875,
  -0.0732421875,
  -0.07373046875,
  -0.0726318359375,
  -0.0718994140625,
  -0.072509765625,
  -0.072509765625,
  -0.0728759765625,
  -0.0726318359375,
  -0.072509765625,
  -0.07275390625,
  -0.072998046875,
  -0.0721435546875,
  -0.0731201171875,
  -0.072265625,
  -0.0731201171875,
  -0.072265625,
  -0.0732421875,
  -0.073974609375,
  -0.072998046875,
  -0.0733642578125,
  -0.072265625,
  -0.07275390625,
  -0.0740966796875,
  -0.0721435546875,
  -0.0718994140625,
  -0.073486328125,
  -0.072265625,
  -0.07275390625,
  -0.0721435546875,
  -0.072998046875,
  -0.073486328125,
  -0.072509765625,
  -0.07373046875,
  -0.07275390625,
  -0.072998046875,
  -0.072021484375,
  -0.0736083984375,
  -0.07275390625,
  -0.0728759765625,
  -0.0731201171875,
  -0.0728759765625,
  -0.072265625,
  -0.0726318359375,
  -0.07275390625,
  -0.072265625,
  -0.0728759765625,
  -0.07275390625,
  -0.0718994140625,
  -0.0726318359375,
  -0.0732421875,
  -0.0733642578125,
  -0.072998046875,
  -0.0731201171875,
  -0.0723876953125,
  -0.07275390625,
  -0.0726318359375,
  -0.0731201171875,
  -0.07275390625,
  -0.07275390625,
  -0.0732421875,
  -0.0726318359375,
  -0.0732421875,
  -0.072021484375,
  -0.0721435546875,
  -0.0726318359375,
  -0.072509765625,
  -0.0733642578125,
  -0.072509765625,
  -0.07177734375,
  -0.072509765625,
  -0.072509765625,
  -0.072998046875,
  -0.0732421875,
  -0.072509765625,
  -0.07275390625,
  -0.0723876953125,
  -0.072998046875,
  -0.0743408203125,
  -0.0740966796875,
  -0.0738525390625,
  -0.0736083984375,
  -0.07275390625,
  -0.072509765625,
  -0.07373046875,
  -0.0732421875,
  -0.0733642578125,
  -0.0733642578125,
  -0.0738525390625,
  -0.0733642578125,
  -0.0733642578125,
  -0.072998046875,
  -0.0736083984375,
  -0.0721435546875,
  -0.072998046875,
  -0.0736083984375,
  -0.072998046875,
  -0.0732421875,
  -0.072021484375,
  -0.0728759765625,
  -0.0726318359375,
  -0.0723876953125,
  -0.0736083984375,
  -0.07275390625,
  -0.0736083984375,
  -0.0733642578125,
  -0.0740966796875,
  -0.072998046875,
  -0.07275390625,
  -0.0716552734375,
  -0.0723876953125,
  -0.072509765625,
  -0.072509765625,
  -0.0721435546875,
  -0.07373046875,
  -0.0721435546875,
  -0.07275390625,
  -0.072265625,
  -0.0721435546875,
  -0.0723876953125,
  -0.07275390625,
  -0.07177734375,
  -0.0733642578125,
  -0.072509765625,
  -0.0736083984375,
  -0.0723876953125,
  -0.072265625,
  -0.0728759765625,
  -0.0733642578125,
  -0.0723876953125,
  -0.0726318359375,
  -0.071533203125,
  -0.0732421875,
  -0.0732421875,
  -0.07275390625,
  -0.0728759765625,
  -0.072265625,
  -0.07275390625,
  -0.0718994140625,
  -0.0732421875,
  -0.0738525390625,
  -0.072509765625,
  -0.072021484375,
  -0.0728759765625,
  -0.0718994140625,
  -0.0721435546875,
  -0.0728759765625,
  -0.0723876953125,
  -0.0716552734375,
  -0.0732421875,
  -0.07275390625,
  -0.072509765625,
  -0.072509765625,
  -0.072998046875,
  -0.0731201171875,
  -0.0732421875,
  -0.0731201171875,
  -0.073486328125,
  -0.0731201171875,
  -0.072509765625,
  -0.07177734375,
  -0.072509765625,
  -0.0728759765625,
  -0.0726318359375,
  -0.072998046875,
  -0.072265625,
  -0.0736083984375,
  -0.0731201171875,
  -0.073974609375,
  -0.0731201171875,
  -0.07275390625,
  -0.0728759765625,
  -0.073486328125,
  -0.0745849609375,
  -0.0726318359375,
  -0.0732421875,
  -0.072998046875,
  -0.073486328125,
  -0.07421875,
  -0.0740966796875,
  -0.073486328125,
  -0.0731201171875,
  -0.073486328125,
  -0.07275390625,
  -0.0728759765625,
  -0.07275390625,
  -0.0731201171875,
  -0.07373046875,
  -0.0721435546875,
  -0.0728759765625,
  -0.072998046875,
  -0.0733642578125,
  -0.0733642578125,
  -0.0728759765625,
  -0.07373046875,
  -0.0733642578125,
  -0.0732421875,
  -0.0728759765625,
  -0.0736083984375,
  -0.0728759765625,
  -0.0731201171875,
  -0.07275390625,
  -0.0736083984375,
  -0.07275390625,
  -0.0731201171875,
  -0.0723876953125,
  -0.0726318359375,
  -0.072509765625,
  -0.07275390625,
  -0.07275390625,
  -0.0733642578125,
  -0.07275390625,
  -0.0726318359375,
  -0.0736083984375,
  -0.0736083984375,
  -0.0726318359375,
  -0.073974609375,
  -0.07275390625,
  -0.0732421875,
  -0.0728759765625,
  -0.072265625,
  -0.0733642578125,
  -0.0723876953125,
  -0.0732421875,
  -0.0732421875,
  -0.07373046875,
  -0.072021484375,
  -0.0723876953125,
  -0.0728759765625,
  -0.0728759765625,
  -0.0733642578125,
  -0.0728759765625,
  -0.0738525390625,
  -0.0736083984375,
  -0.0733642578125,
  -0.072265625,
  -0.072509765625,
  -0.072509765625,
  -0.0723876953125,
  -0.073974609375,
  -0.0728759765625,
  -0.0726318359375,
  -0.073486328125,
  -0.0740966796875,
  -0.0736083984375,
  -0.072998046875,
  -0.0736083984375,
  -0.073974609375,
  -0.0731201171875,
  -0.0733642578125,
  -0.0728759765625,
  -0.0728759765625,
  -0.0728759765625,
  -0.0732421875,
  -0.0740966796875,
  -0.0726318359375,
  -0.0728759765625,
  -0.0738525390625,
  -0.0731201171875,
  -0.0736083984375,
  -0.07275390625,
  -0.073974609375,
  -0.07275390625,
  -0.07275390625,
  -0.072998046875,
  -0.0728759765625,
  -0.07373046875,
  -0.0731201171875,
  -0.0723876953125,
  -0.072509765625,
  -0.07275390625,
  -0.072509765625,
  -0.0732421875,
  -0.0723876953125,
  -0.07275390625,
  -0.0736083984375,
  -0.073486328125,
  -0.072265625,
  -0.0738525390625,
  -0.0728759765625,
  -0.0726318359375,
  -0.07275390625,
  -0.073486328125,
  -0.0743408203125,
  -0.072509765625,
  -0.07373046875,
  -0.0732421875,
  -0.07177734375,
  -0.072998046875,
  -0.0728759765625,
  -0.0731201171875,
  -0.072998046875,
  -0.0732421875,
  -0.0731201171875,
  -0.0732421875,
  -0.073486328125,
  -0.072998046875,
  -0.0732421875,
  -0.0728759765625,
  -0.0726318359375,
  -0.0733642578125,
  -0.0738525390625,
  -0.072265625,
  -0.072265625,
  -0.0728759765625,
  -0.072998046875,
  -0.072998046875,
  -0.07275390625,
  -0.0728759765625,
  -0.0728759765625,
  -0.0733642578125,
  -0.071533203125,
  -0.07275390625,
  -0.072509765625,
  -0.072998046875,
  -0.072265625,
  -0.073486328125,
  -0.0733642578125,
  -0.0732421875,
  -0.0728759765625,
  -0.073486328125,
  -0.0736083984375,
  -0.0723876953125,
  -0.072998046875,
  -0.0732421875,
  -0.072998046875,
  -0.07373046875,
  -0.0738525390625,
  -0.0726318359375,
  -0.072265625,
  -0.0732421875,
  -0.072998046875,
  -0.0726318359375,
  -0.0718994140625,
  -0.07373046875,
  -0.0732421875,
  -0.073486328125,
  -0.072998046875,
  -0.073486328125,
  -0.07275390625,
  -0.07275390625,
  -0.0726318359375,
  -0.0731201171875]

In [40]:
ay = [0.501220703125, 0.50244140625, 0.501953125, 0.501953125, 0.50146484375, 0.5018310546875, 0.501708984375, 0.5018310546875, 0.5020751953125, 0.5018310546875, 0.50146484375, 0.50146484375, 0.50244140625, 0.502685546875, 0.50244140625, 0.502197265625, 0.5015869140625, 0.501220703125, 0.5015869140625, 0.502197265625, 0.501220703125, 0.501220703125, 0.5015869140625, 0.502685546875, 0.501708984375, 0.501708984375, 0.5013427734375, 0.5020751953125, 0.502197265625, 0.502197265625, 0.5013427734375, 0.5009765625, 0.50146484375, 0.50244140625, 0.50048828125, 0.5025634765625, 0.50146484375, 0.5015869140625, 0.50244140625, 0.501708984375, 0.501953125, 0.50244140625, 0.5015869140625, 0.501220703125, 0.5009765625, 0.501953125, 0.5020751953125, 0.5030517578125, 0.50341796875, 0.5028076171875, 0.502197265625, 0.5020751953125, 0.5025634765625, 0.5015869140625, 0.5032958984375, 0.501953125, 0.503173828125, 0.5009765625, 0.503173828125, 0.50146484375, 0.5023193359375, 0.50244140625, 0.5008544921875, 0.501708984375, 0.501708984375, 0.501708984375, 0.50146484375, 0.501953125, 0.5020751953125, 0.50244140625, 0.5025634765625, 0.502197265625, 0.5010986328125, 0.50244140625, 0.501220703125, 0.5035400390625, 0.5020751953125, 0.50244140625, 0.5030517578125, 0.5023193359375, 0.502685546875, 0.5025634765625, 0.501953125, 0.502685546875, 0.5020751953125, 0.5018310546875, 0.5025634765625, 0.501953125, 0.50244140625, 0.5023193359375, 0.5006103515625, 0.5018310546875, 0.5023193359375, 0.50146484375, 0.501953125, 0.502197265625, 0.502197265625, 0.5023193359375, 0.501708984375, 0.502685546875, 0.502197265625, 0.5025634765625, 0.5025634765625, 0.5018310546875, 0.5020751953125, 0.50244140625, 0.501708984375, 0.50244140625, 0.502197265625, 0.5018310546875, 0.5025634765625, 0.5015869140625, 0.50244140625, 0.5025634765625, 0.5013427734375, 0.502197265625, 0.501708984375, 0.501220703125, 0.50146484375, 0.5020751953125, 0.5015869140625, 0.501953125, 0.5020751953125, 0.50146484375, 0.5018310546875, 0.5018310546875, 0.5018310546875, 0.5023193359375, 0.501220703125, 0.5023193359375, 0.5015869140625, 0.50146484375, 0.5020751953125, 0.5030517578125, 0.5025634765625, 0.5013427734375, 0.5015869140625, 0.5013427734375, 0.50244140625, 0.5028076171875, 0.50244140625, 0.5023193359375, 0.501708984375, 0.501708984375, 0.501708984375, 0.50146484375, 0.501953125, 0.502197265625, 0.501708984375, 0.501953125, 0.5010986328125, 0.5020751953125, 0.5030517578125, 0.50244140625, 0.5018310546875, 0.502197265625, 0.501220703125, 0.502197265625, 0.501953125, 0.501953125, 0.501953125, 0.50244140625, 0.5020751953125, 0.5023193359375, 0.5025634765625, 0.5020751953125, 0.502685546875, 0.5006103515625, 0.502197265625, 0.50244140625, 0.501708984375, 0.50244140625, 0.5030517578125, 0.5030517578125, 0.5025634765625, 0.5023193359375, 0.5018310546875, 0.5010986328125, 0.5029296875, 0.50244140625, 0.5020751953125, 0.502197265625, 0.501708984375, 0.5025634765625, 0.5029296875, 0.501708984375, 0.50244140625, 0.501708984375, 0.5028076171875, 0.5018310546875, 0.5023193359375, 0.5029296875, 0.501708984375, 0.5015869140625, 0.5013427734375, 0.5013427734375, 0.5018310546875, 0.5013427734375, 0.5013427734375, 0.501708984375, 0.5020751953125, 0.5023193359375, 0.5028076171875, 0.50244140625, 0.5018310546875, 0.50146484375, 0.502685546875, 0.501953125, 0.501708984375, 0.501708984375, 0.502685546875, 0.50146484375, 0.5025634765625, 0.5013427734375, 0.50244140625, 0.50244140625, 0.501953125, 0.50244140625, 0.5020751953125, 0.5028076171875, 0.5020751953125, 0.502685546875, 0.5025634765625, 0.5023193359375, 0.5023193359375, 0.502197265625, 0.5023193359375, 0.5010986328125, 0.502197265625, 0.502197265625, 0.501953125, 0.5009765625, 0.5013427734375, 0.5018310546875, 0.5020751953125, 0.50146484375, 0.50146484375, 0.5018310546875, 0.50146484375, 0.501708984375, 0.502197265625, 0.502197265625, 0.5018310546875, 0.50244140625, 0.501953125, 0.5015869140625, 0.5015869140625, 0.5015869140625, 0.50244140625, 0.500732421875, 0.502685546875, 0.501708984375, 0.501953125, 0.50244140625, 0.502197265625, 0.5023193359375, 0.501708984375, 0.5018310546875, 0.5008544921875, 0.502197265625, 0.5020751953125, 0.5025634765625, 0.5025634765625, 0.50146484375, 0.5013427734375, 0.5015869140625, 0.5025634765625, 0.50146484375, 0.501953125, 0.5006103515625, 0.501220703125, 0.5020751953125, 0.5006103515625, 0.5018310546875, 0.5020751953125, 0.5018310546875, 0.5018310546875, 0.5023193359375, 0.5015869140625, 0.5032958984375, 0.50244140625, 0.501953125, 0.501220703125, 0.501708984375, 0.501220703125, 0.502685546875, 0.502197265625, 0.5025634765625, 0.5020751953125, 0.5020751953125, 0.5020751953125, 0.501953125, 0.501708984375, 0.50146484375, 0.501708984375, 0.502197265625, 0.502197265625, 0.5010986328125, 0.5023193359375, 0.5018310546875, 0.5006103515625, 0.5013427734375, 0.5028076171875, 0.5023193359375, 0.5018310546875, 0.5018310546875, 0.5028076171875, 0.502197265625, 0.50146484375, 0.50048828125, 0.501953125, 0.5028076171875, 0.5020751953125, 0.50146484375, 0.502685546875, 0.5023193359375, 0.501953125, 0.5023193359375, 0.5015869140625, 0.501953125, 0.5023193359375, 0.50146484375, 0.50146484375, 0.501220703125, 0.5015869140625, 0.5023193359375, 0.5015869140625, 0.5018310546875, 0.50244140625, 0.502685546875, 0.5023193359375, 0.50146484375, 0.5023193359375, 0.5025634765625, 0.501953125, 0.5008544921875, 0.501708984375, 0.5018310546875, 0.50146484375, 0.5018310546875, 0.501708984375, 0.50244140625, 0.501953125, 0.5003662109375, 0.5013427734375, 0.5008544921875, 0.5018310546875, 0.5015869140625, 0.502197265625, 0.5023193359375, 0.501708984375, 0.5015869140625, 0.501708984375, 0.5025634765625, 0.50244140625, 0.5023193359375, 0.501708984375, 0.501953125, 0.5020751953125, 0.5018310546875, 0.5018310546875, 0.501220703125, 0.502685546875, 0.502197265625, 0.501708984375, 0.501708984375, 0.5029296875, 0.501220703125, 0.50244140625, 0.5013427734375, 0.5020751953125, 0.501708984375, 0.5018310546875, 0.5013427734375, 0.50146484375, 0.5009765625, 0.5023193359375, 0.502685546875, 0.5025634765625, 0.5018310546875, 0.501953125, 0.5030517578125, 0.5013427734375, 0.5023193359375, 0.5023193359375, 0.501708984375, 0.50146484375, 0.5020751953125, 0.502197265625, 0.501953125, 0.50244140625, 0.5018310546875, 0.50146484375, 0.5010986328125, 0.5001220703125, 0.5001220703125, 0.5018310546875, 0.5020751953125, 0.5023193359375, 0.501708984375, 0.502197265625, 0.5020751953125, 0.502685546875, 0.50146484375, 0.501708984375, 0.5020751953125, 0.5015869140625, 0.500732421875, 0.5018310546875, 0.5023193359375, 0.501708984375, 0.5028076171875, 0.50244140625, 0.5018310546875, 0.502685546875, 0.5025634765625, 0.5020751953125, 0.5023193359375, 0.5020751953125, 0.5020751953125, 0.5008544921875, 0.5029296875, 0.5018310546875, 0.501953125, 0.501953125, 0.50146484375, 0.5020751953125, 0.5003662109375, 0.5009765625, 0.501220703125, 0.501953125, 0.5023193359375, 0.5018310546875, 0.502197265625, 0.502197265625, 0.5025634765625, 0.501953125, 0.501220703125, 0.501708984375, 0.5028076171875, 0.5015869140625, 0.501708984375, 0.500732421875, 0.5013427734375, 0.50146484375, 0.5006103515625, 0.50244140625, 0.50244140625, 0.501953125, 0.50244140625, 0.5028076171875, 0.501953125, 0.501953125, 0.50146484375, 0.502197265625, 0.5018310546875, 0.5025634765625, 0.5015869140625, 0.5020751953125, 0.50146484375, 0.502197265625, 0.5020751953125, 0.50244140625, 0.501953125, 0.501953125, 0.5032958984375, 0.5015869140625, 0.50244140625, 0.5018310546875, 0.501708984375, 0.5008544921875, 0.502685546875, 0.5020751953125, 0.5008544921875, 0.5015869140625, 0.5013427734375, 0.501708984375, 0.501953125, 0.5013427734375, 0.5029296875, 0.5015869140625, 0.5025634765625, 0.502685546875, 0.501708984375, 0.501953125, 0.5018310546875, 0.502197265625, 0.503173828125, 0.5018310546875, 0.50244140625, 0.5023193359375, 0.5025634765625, 0.50244140625, 0.5015869140625, 0.502685546875, 0.5020751953125, 0.5013427734375, 0.5029296875, 0.5015869140625, 0.502197265625]

In [42]:
az = [0.9022216796875, 0.9012451171875, 0.899658203125, 0.9007568359375, 0.9024658203125, 0.9017333984375, 0.90283203125, 0.9013671875, 0.90185546875, 0.9002685546875, 0.8995361328125, 0.9012451171875, 0.9000244140625, 0.9013671875, 0.899658203125, 0.9034423828125, 0.9017333984375, 0.9010009765625, 0.903076171875, 0.9029541015625, 0.9012451171875, 0.9024658203125, 0.9022216796875, 0.9024658203125, 0.90283203125, 0.903076171875, 0.90283203125, 0.9024658203125, 0.901611328125, 0.90087890625, 0.9017333984375, 0.9014892578125, 0.90185546875, 0.9002685546875, 0.900634765625, 0.90185546875, 0.9005126953125, 0.9010009765625, 0.900634765625, 0.9013671875, 0.9005126953125, 0.900390625, 0.902587890625, 0.9017333984375, 0.8995361328125, 0.9010009765625, 0.9010009765625, 0.9014892578125, 0.90234375, 0.901611328125, 0.900390625, 0.901611328125, 0.9014892578125, 0.902099609375, 0.90185546875, 0.90234375, 0.9014892578125, 0.901123046875, 0.90185546875, 0.9034423828125, 0.8994140625, 0.902099609375, 0.903076171875, 0.9017333984375, 0.902587890625, 0.900390625, 0.9012451171875, 0.900146484375, 0.900146484375, 0.903076171875, 0.9010009765625, 0.901611328125, 0.90283203125, 0.901611328125, 0.90185546875, 0.89990234375, 0.90234375, 0.9024658203125, 0.903076171875, 0.9027099609375, 0.9024658203125, 0.902099609375, 0.9024658203125, 0.9019775390625, 0.9014892578125, 0.900390625, 0.902587890625, 0.90185546875, 0.900390625, 0.902099609375, 0.901123046875, 0.9024658203125, 0.9022216796875, 0.9010009765625, 0.9007568359375, 0.9017333984375, 0.9002685546875, 0.9013671875, 0.9022216796875, 0.9022216796875, 0.89990234375, 0.902099609375, 0.902099609375, 0.89990234375, 0.900146484375, 0.900390625, 0.9022216796875, 0.900146484375, 0.9022216796875, 0.9033203125, 0.900146484375, 0.9022216796875, 0.9013671875, 0.905029296875, 0.9010009765625, 0.9007568359375, 0.901123046875, 0.90087890625, 0.901611328125, 0.9013671875, 0.9000244140625, 0.9002685546875, 0.9027099609375, 0.90234375, 0.9014892578125, 0.900634765625, 0.901123046875, 0.9010009765625, 0.901123046875, 0.9010009765625, 0.9013671875, 0.90087890625, 0.9005126953125, 0.9007568359375, 0.901123046875, 0.8980712890625, 0.9012451171875, 0.9002685546875, 0.9013671875, 0.8992919921875, 0.9013671875, 0.9010009765625, 0.9013671875, 0.90185546875, 0.9010009765625, 0.9014892578125, 0.9007568359375, 0.9010009765625, 0.901611328125, 0.8994140625, 0.9014892578125, 0.9002685546875, 0.9010009765625, 0.9012451171875, 0.9002685546875, 0.9017333984375, 0.9027099609375, 0.9013671875, 0.9010009765625, 0.9012451171875, 0.9039306640625, 0.90234375, 0.9012451171875, 0.901123046875, 0.901123046875, 0.900146484375, 0.90283203125, 0.9012451171875, 0.90185546875, 0.9007568359375, 0.9007568359375, 0.900634765625, 0.902587890625, 0.9024658203125, 0.9002685546875, 0.900634765625, 0.9005126953125, 0.90283203125, 0.90087890625, 0.9013671875, 0.9000244140625, 0.9019775390625, 0.900390625, 0.901611328125, 0.902587890625, 0.901611328125, 0.901123046875, 0.8997802734375, 0.900634765625, 0.900634765625, 0.90185546875, 0.90283203125, 0.901611328125, 0.902587890625, 0.903076171875, 0.9000244140625, 0.9010009765625, 0.901123046875, 0.9010009765625, 0.901123046875, 0.9005126953125, 0.9012451171875, 0.9012451171875, 0.9019775390625, 0.9022216796875, 0.9005126953125, 0.9010009765625, 0.902587890625, 0.9017333984375, 0.901123046875, 0.9013671875, 0.90185546875, 0.89990234375, 0.900390625, 0.9007568359375, 0.90185546875, 0.900390625, 0.9007568359375, 0.899169921875, 0.9017333984375, 0.9027099609375, 0.900634765625, 0.900634765625, 0.901123046875, 0.9017333984375, 0.9022216796875, 0.9019775390625, 0.9022216796875, 0.90087890625, 0.901123046875, 0.9014892578125, 0.9024658203125, 0.90087890625, 0.90234375, 0.9013671875, 0.9019775390625, 0.901611328125, 0.9019775390625, 0.90185546875, 0.900634765625, 0.9019775390625, 0.9019775390625, 0.9017333984375, 0.90234375, 0.9019775390625, 0.9017333984375, 0.9005126953125, 0.902099609375, 0.902099609375, 0.9005126953125, 0.901123046875, 0.9013671875, 0.900390625, 0.9022216796875, 0.9027099609375, 0.90087890625, 0.9039306640625, 0.899169921875, 0.9017333984375, 0.90087890625, 0.902099609375, 0.901611328125, 0.9024658203125, 0.9005126953125, 0.9012451171875, 0.901123046875, 0.9022216796875, 0.9019775390625, 0.9022216796875, 0.9013671875, 0.90087890625, 0.9013671875, 0.90185546875, 0.900634765625, 0.8995361328125, 0.8988037109375, 0.9013671875, 0.9014892578125, 0.9013671875, 0.90087890625, 0.9014892578125, 0.9024658203125, 0.9010009765625, 0.90087890625, 0.9014892578125, 0.900146484375, 0.901611328125, 0.9005126953125, 0.9012451171875, 0.901611328125, 0.9014892578125, 0.9012451171875, 0.9005126953125, 0.9005126953125, 0.9013671875, 0.9013671875, 0.9014892578125, 0.902099609375, 0.9013671875, 0.9005126953125, 0.900634765625, 0.9000244140625, 0.90185546875, 0.9013671875, 0.9029541015625, 0.901611328125, 0.9014892578125, 0.8997802734375, 0.89990234375, 0.90185546875, 0.9005126953125, 0.90185546875, 0.90185546875, 0.90185546875, 0.9022216796875, 0.9005126953125, 0.8995361328125, 0.90087890625, 0.9013671875, 0.9014892578125, 0.9027099609375, 0.9005126953125, 0.901611328125, 0.9012451171875, 0.9010009765625, 0.9010009765625, 0.9019775390625, 0.9014892578125, 0.9013671875, 0.9022216796875, 0.8995361328125, 0.8994140625, 0.9002685546875, 0.901123046875, 0.9027099609375, 0.9022216796875, 0.90087890625, 0.900634765625, 0.9010009765625, 0.901123046875, 0.900390625, 0.9013671875, 0.9017333984375, 0.901611328125, 0.90087890625, 0.9007568359375, 0.9027099609375, 0.9017333984375, 0.9007568359375, 0.9005126953125, 0.90087890625, 0.9005126953125, 0.9019775390625, 0.902587890625, 0.902099609375, 0.900634765625, 0.9013671875, 0.902587890625, 0.9017333984375, 0.9031982421875, 0.9019775390625, 0.9014892578125, 0.9024658203125, 0.9017333984375, 0.90087890625, 0.90087890625, 0.9010009765625, 0.901123046875, 0.9014892578125, 0.902099609375, 0.900146484375, 0.9017333984375, 0.9019775390625, 0.899658203125, 0.9005126953125, 0.90185546875, 0.90234375, 0.9013671875, 0.9013671875, 0.900634765625, 0.899658203125, 0.9005126953125, 0.901611328125, 0.902099609375, 0.9005126953125, 0.900634765625, 0.9014892578125, 0.9014892578125, 0.9007568359375, 0.9019775390625, 0.900634765625, 0.90185546875, 0.902099609375, 0.90283203125, 0.90087890625, 0.90087890625, 0.9024658203125, 0.9005126953125, 0.9012451171875, 0.9017333984375, 0.90185546875, 0.901123046875, 0.9013671875, 0.9010009765625, 0.9014892578125, 0.9019775390625, 0.901123046875, 0.9005126953125, 0.901123046875, 0.900146484375, 0.9019775390625, 0.901611328125, 0.901123046875, 0.9019775390625, 0.9027099609375, 0.9017333984375, 0.903076171875, 0.9010009765625, 0.9013671875, 0.899658203125, 0.9014892578125, 0.9017333984375, 0.9017333984375, 0.9007568359375, 0.9000244140625, 0.899658203125, 0.9019775390625, 0.901123046875, 0.9034423828125, 0.9017333984375, 0.9017333984375, 0.903076171875, 0.901123046875, 0.9000244140625, 0.9027099609375, 0.9007568359375, 0.9029541015625, 0.90185546875, 0.9017333984375, 0.899658203125, 0.9013671875, 0.900390625, 0.8997802734375, 0.900634765625, 0.9012451171875, 0.9010009765625, 0.901123046875, 0.902587890625, 0.9017333984375, 0.9005126953125, 0.9022216796875, 0.899658203125, 0.9012451171875, 0.9017333984375, 0.9022216796875, 0.9013671875, 0.9012451171875, 0.9010009765625, 0.9013671875, 0.9019775390625, 0.902099609375, 0.90185546875, 0.9017333984375, 0.900146484375, 0.9014892578125, 0.8997802734375, 0.9019775390625, 0.90087890625, 0.90087890625, 0.9010009765625, 0.9019775390625, 0.9000244140625, 0.90234375, 0.9010009765625, 0.902587890625, 0.902099609375, 0.900634765625, 0.9007568359375, 0.9012451171875, 0.901123046875, 0.90087890625, 0.90087890625, 0.9007568359375, 0.900634765625, 0.90087890625, 0.902587890625, 0.902099609375, 0.89990234375, 0.900634765625, 0.9007568359375, 0.9036865234375, 0.9024658203125, 0.8995361328125, 0.9005126953125, 0.900634765625, 0.900146484375, 0.902099609375, 0.90185546875, 0.9014892578125, 0.901123046875]

In [43]:
gz = [2.2125244140625, 2.25830078125, 2.2430419921875, 2.2430419921875, 2.2125244140625, 2.227783203125, 2.2430419921875, 2.227783203125, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.2430419921875, 2.25830078125, 2.2125244140625, 2.2125244140625, 2.197265625, 2.227783203125, 2.197265625, 2.227783203125, 2.2430419921875, 2.227783203125, 2.2125244140625, 2.197265625, 2.2125244140625, 2.227783203125, 2.227783203125, 2.197265625, 2.197265625, 2.197265625, 2.2430419921875, 2.1820068359375, 2.2430419921875, 2.25830078125, 2.227783203125, 2.227783203125, 2.2125244140625, 2.227783203125, 2.197265625, 2.197265625, 2.227783203125, 2.2430419921875, 2.197265625, 2.1820068359375, 2.1514892578125, 2.1820068359375, 2.227783203125, 2.2125244140625, 2.197265625, 2.227783203125, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.2430419921875, 2.227783203125, 2.2125244140625, 2.197265625, 2.227783203125, 2.227783203125, 2.197265625, 2.197265625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.227783203125, 2.227783203125, 2.2735595703125, 2.227783203125, 2.2430419921875, 2.1820068359375, 2.1820068359375, 2.227783203125, 2.197265625, 2.2430419921875, 2.2430419921875, 2.2125244140625, 2.1820068359375, 2.197265625, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.2430419921875, 2.2430419921875, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.1820068359375, 2.197265625, 2.2430419921875, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.1820068359375, 2.2125244140625, 2.227783203125, 2.227783203125, 2.2735595703125, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.227783203125, 2.2430419921875, 2.2125244140625, 2.227783203125, 2.197265625, 2.197265625, 2.2125244140625, 2.2125244140625, 2.1514892578125, 2.2125244140625, 2.227783203125, 2.227783203125, 2.1820068359375, 2.227783203125, 2.2125244140625, 2.227783203125, 2.25830078125, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.25830078125, 2.2430419921875, 2.2125244140625, 2.197265625, 2.227783203125, 2.227783203125, 2.197265625, 2.197265625, 2.25830078125, 2.2430419921875, 2.2735595703125, 2.227783203125, 2.227783203125, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.197265625, 2.227783203125, 2.227783203125, 2.2430419921875, 2.227783203125, 2.227783203125, 2.197265625, 2.166748046875, 2.227783203125, 2.227783203125, 2.197265625, 2.1820068359375, 2.2125244140625, 2.1820068359375, 2.2125244140625, 2.197265625, 2.2430419921875, 2.227783203125, 2.197265625, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.2430419921875, 2.227783203125, 2.166748046875, 2.2125244140625, 2.2430419921875, 2.2430419921875, 2.2430419921875, 2.2125244140625, 2.197265625, 2.197265625, 2.227783203125, 2.2430419921875, 2.2430419921875, 2.2125244140625, 2.1820068359375, 2.2430419921875, 2.1820068359375, 2.1820068359375, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.2430419921875, 2.2125244140625, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.166748046875, 2.227783203125, 2.2125244140625, 2.197265625, 2.197265625, 2.166748046875, 2.2430419921875, 2.2125244140625, 2.25830078125, 2.227783203125, 2.227783203125, 2.2125244140625, 2.197265625, 2.197265625, 2.2430419921875, 2.2125244140625, 2.2125244140625, 2.197265625, 2.197265625, 2.2735595703125, 2.25830078125, 2.227783203125, 2.197265625, 2.1820068359375, 2.2125244140625, 2.2125244140625, 2.1820068359375, 2.2125244140625, 2.1820068359375, 2.1820068359375, 2.2125244140625, 2.197265625, 2.2430419921875, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.197265625, 2.197265625, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.227783203125, 2.2430419921875, 2.2125244140625, 2.2430419921875, 2.227783203125, 2.227783203125, 2.197265625, 2.197265625, 2.2125244140625, 2.227783203125, 2.227783203125, 2.25830078125, 2.197265625, 2.1820068359375, 2.197265625, 2.197265625, 2.227783203125, 2.197265625, 2.197265625, 2.1820068359375, 2.197265625, 2.227783203125, 2.227783203125, 2.2125244140625, 2.197265625, 2.1820068359375, 2.1820068359375, 2.197265625, 2.197265625, 2.227783203125, 2.197265625, 2.197265625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.1820068359375, 2.197265625, 2.1514892578125, 2.1820068359375, 2.2125244140625, 2.197265625, 2.2125244140625, 2.197265625, 2.1820068359375, 2.227783203125, 2.197265625, 2.197265625, 2.197265625, 2.166748046875, 2.2430419921875, 2.197265625, 2.2125244140625, 2.2125244140625, 2.1820068359375, 2.1820068359375, 2.2430419921875, 2.25830078125, 2.2125244140625, 2.227783203125, 2.227783203125, 2.2125244140625, 2.25830078125, 2.2125244140625, 2.197265625, 2.2735595703125, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.197265625, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.197265625, 2.1514892578125, 2.227783203125, 2.2735595703125, 2.2125244140625, 2.166748046875, 2.166748046875, 2.1820068359375, 2.227783203125, 2.1820068359375, 2.227783203125, 2.2125244140625, 2.227783203125, 2.227783203125, 2.227783203125, 2.227783203125, 2.227783203125, 2.1820068359375, 2.2125244140625, 2.197265625, 2.1820068359375, 2.197265625, 2.166748046875, 2.1820068359375, 2.197265625, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.197265625, 2.2125244140625, 2.197265625, 2.227783203125, 2.197265625, 2.197265625, 2.2125244140625, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.197265625, 2.197265625, 2.166748046875, 2.197265625, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.197265625, 2.227783203125, 2.2125244140625, 2.227783203125, 2.227783203125, 2.2125244140625, 2.197265625, 2.166748046875, 2.197265625, 2.227783203125, 2.227783203125, 2.2125244140625, 2.197265625, 2.166748046875, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.227783203125, 2.197265625, 2.2125244140625, 2.2125244140625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.197265625, 2.227783203125, 2.227783203125, 2.227783203125, 2.197265625, 2.197265625, 2.197265625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.197265625, 2.197265625, 2.2430419921875, 2.2430419921875, 2.2125244140625, 2.1820068359375, 2.227783203125, 2.227783203125, 2.1820068359375, 2.166748046875, 2.197265625, 2.166748046875, 2.197265625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.2430419921875, 2.197265625, 2.197265625, 2.197265625, 2.197265625, 2.2125244140625, 2.166748046875, 2.227783203125, 2.2430419921875, 2.227783203125, 2.197265625, 2.1820068359375, 2.2125244140625, 2.197265625, 2.2125244140625, 2.197265625, 2.166748046875, 2.166748046875, 2.2125244140625, 2.2125244140625, 2.197265625, 2.1820068359375, 2.1820068359375, 2.1514892578125, 2.1820068359375, 2.197265625, 2.227783203125, 2.1820068359375, 2.197265625, 2.2125244140625, 2.2125244140625, 2.2125244140625, 2.1820068359375, 2.227783203125, 2.2430419921875, 2.197265625, 2.1820068359375, 2.227783203125, 2.227783203125, 2.2430419921875, 2.2430419921875, 2.227783203125, 2.1820068359375, 2.13623046875, 2.2125244140625, 2.227783203125, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.197265625, 2.2430419921875, 2.1820068359375, 2.197265625, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.2430419921875, 2.2125244140625, 2.197265625, 2.2125244140625, 2.2430419921875, 2.2430419921875, 2.2430419921875, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.25830078125, 2.2430419921875, 2.2430419921875, 2.197265625, 2.2125244140625, 2.2125244140625, 2.227783203125, 2.2430419921875, 2.2430419921875, 2.227783203125, 2.227783203125, 2.1820068359375, 2.1820068359375, 2.1820068359375]

In [44]:
gy = [0.6256103515625, 0.6103515625, 0.6256103515625, 0.6256103515625, 0.67138671875, 0.640869140625, 0.67138671875, 0.6866455078125, 0.701904296875, 0.67138671875, 0.7171630859375, 0.6561279296875, 0.640869140625, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.6561279296875, 0.6866455078125, 0.67138671875, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.6256103515625, 0.701904296875, 0.6561279296875, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.67138671875, 0.6866455078125, 0.7171630859375, 0.67138671875, 0.67138671875, 0.701904296875, 0.6866455078125, 0.67138671875, 0.6256103515625, 0.6866455078125, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.640869140625, 0.67138671875, 0.67138671875, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.640869140625, 0.6866455078125, 0.67138671875, 0.67138671875, 0.6561279296875, 0.6866455078125, 0.6103515625, 0.6561279296875, 0.6561279296875, 0.640869140625, 0.640869140625, 0.640869140625, 0.701904296875, 0.6866455078125, 0.6866455078125, 0.6866455078125, 0.67138671875, 0.701904296875, 0.6561279296875, 0.6866455078125, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.701904296875, 0.67138671875, 0.640869140625, 0.6103515625, 0.67138671875, 0.67138671875, 0.7171630859375, 0.640869140625, 0.6561279296875, 0.701904296875, 0.640869140625, 0.640869140625, 0.701904296875, 0.67138671875, 0.640869140625, 0.640869140625, 0.5950927734375, 0.6256103515625, 0.6561279296875, 0.6561279296875, 0.6866455078125, 0.67138671875, 0.701904296875, 0.640869140625, 0.640869140625, 0.640869140625, 0.640869140625, 0.67138671875, 0.701904296875, 0.640869140625, 0.6256103515625, 0.640869140625, 0.6561279296875, 0.6866455078125, 0.6256103515625, 0.640869140625, 0.6866455078125, 0.6561279296875, 0.6866455078125, 0.6561279296875, 0.6256103515625, 0.640869140625, 0.6866455078125, 0.67138671875, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.640869140625, 0.640869140625, 0.6103515625, 0.640869140625, 0.640869140625, 0.6561279296875, 0.6561279296875, 0.6561279296875, 0.6256103515625, 0.6561279296875, 0.67138671875, 0.6866455078125, 0.67138671875, 0.6561279296875, 0.67138671875, 0.67138671875, 0.6866455078125, 0.6561279296875, 0.6866455078125, 0.6561279296875, 0.6561279296875, 0.6561279296875, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.6561279296875, 0.701904296875, 0.6866455078125, 0.6256103515625, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.701904296875, 0.67138671875, 0.67138671875, 0.701904296875, 0.6561279296875, 0.6256103515625, 0.67138671875, 0.67138671875, 0.6866455078125, 0.701904296875, 0.6561279296875, 0.6561279296875, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.67138671875, 0.67138671875, 0.6866455078125, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.701904296875, 0.732421875, 0.701904296875, 0.6256103515625, 0.6256103515625, 0.6561279296875, 0.67138671875, 0.67138671875, 0.6561279296875, 0.6561279296875, 0.701904296875, 0.6561279296875, 0.67138671875, 0.640869140625, 0.6866455078125, 0.6256103515625, 0.6561279296875, 0.67138671875, 0.640869140625, 0.701904296875, 0.6561279296875, 0.640869140625, 0.640869140625, 0.640869140625, 0.640869140625, 0.6561279296875, 0.640869140625, 0.6561279296875, 0.640869140625, 0.6561279296875, 0.6256103515625, 0.67138671875, 0.7171630859375, 0.701904296875, 0.6866455078125, 0.640869140625, 0.6561279296875, 0.6561279296875, 0.6561279296875, 0.640869140625, 0.6256103515625, 0.6866455078125, 0.6866455078125, 0.7171630859375, 0.6866455078125, 0.640869140625, 0.640869140625, 0.6561279296875, 0.6561279296875, 0.701904296875, 0.7476806640625, 0.7171630859375, 0.6866455078125, 0.6561279296875, 0.701904296875, 0.6866455078125, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.67138671875, 0.6866455078125, 0.6561279296875, 0.6866455078125, 0.640869140625, 0.6866455078125, 0.7171630859375, 0.6866455078125, 0.6561279296875, 0.640869140625, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.67138671875, 0.701904296875, 0.6866455078125, 0.6256103515625, 0.6103515625, 0.6561279296875, 0.67138671875, 0.6866455078125, 0.701904296875, 0.6866455078125, 0.67138671875, 0.640869140625, 0.640869140625, 0.640869140625, 0.640869140625, 0.6866455078125, 0.6866455078125, 0.6256103515625, 0.6561279296875, 0.640869140625, 0.67138671875, 0.640869140625, 0.6561279296875, 0.640869140625, 0.7171630859375, 0.7476806640625, 0.732421875, 0.67138671875, 0.67138671875, 0.6561279296875, 0.6866455078125, 0.6561279296875, 0.6561279296875, 0.640869140625, 0.6866455078125, 0.701904296875, 0.6866455078125, 0.701904296875, 0.67138671875, 0.640869140625, 0.640869140625, 0.67138671875, 0.6561279296875, 0.640869140625, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.6103515625, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.640869140625, 0.640869140625, 0.640869140625, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.67138671875, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.67138671875, 0.701904296875, 0.6866455078125, 0.6866455078125, 0.67138671875, 0.701904296875, 0.640869140625, 0.640869140625, 0.640869140625, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.6866455078125, 0.67138671875, 0.640869140625, 0.640869140625, 0.701904296875, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.640869140625, 0.6103515625, 0.6561279296875, 0.67138671875, 0.701904296875, 0.6866455078125, 0.6866455078125, 0.67138671875, 0.6866455078125, 0.6103515625, 0.6561279296875, 0.640869140625, 0.67138671875, 0.7171630859375, 0.67138671875, 0.67138671875, 0.6866455078125, 0.67138671875, 0.6866455078125, 0.67138671875, 0.6561279296875, 0.640869140625, 0.6256103515625, 0.640869140625, 0.67138671875, 0.640869140625, 0.640869140625, 0.67138671875, 0.67138671875, 0.6561279296875, 0.640869140625, 0.67138671875, 0.67138671875, 0.640869140625, 0.6256103515625, 0.6103515625, 0.640869140625, 0.6866455078125, 0.67138671875, 0.67138671875, 0.6561279296875, 0.6866455078125, 0.701904296875, 0.7171630859375, 0.67138671875, 0.67138671875, 0.6866455078125, 0.701904296875, 0.640869140625, 0.6103515625, 0.6256103515625, 0.6103515625, 0.6561279296875, 0.6103515625, 0.67138671875, 0.701904296875, 0.6561279296875, 0.6561279296875, 0.6561279296875, 0.7171630859375, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.67138671875, 0.6561279296875, 0.6561279296875, 0.7171630859375, 0.7171630859375, 0.67138671875, 0.640869140625, 0.6256103515625, 0.640869140625, 0.6256103515625, 0.6103515625, 0.640869140625, 0.640869140625, 0.6866455078125, 0.67138671875, 0.701904296875, 0.6561279296875, 0.640869140625, 0.6866455078125, 0.6866455078125, 0.67138671875, 0.6561279296875, 0.6256103515625, 0.640869140625, 0.640869140625, 0.640869140625, 0.640869140625, 0.6561279296875, 0.640869140625, 0.6866455078125, 0.6866455078125, 0.67138671875, 0.67138671875, 0.67138671875, 0.640869140625, 0.6561279296875, 0.701904296875, 0.640869140625, 0.6103515625, 0.7171630859375, 0.67138671875, 0.640869140625, 0.6256103515625, 0.6561279296875, 0.640869140625, 0.6866455078125, 0.6561279296875, 0.6866455078125, 0.6561279296875, 0.6561279296875, 0.7171630859375, 0.6561279296875, 0.640869140625, 0.640869140625, 0.6256103515625, 0.6561279296875, 0.6866455078125, 0.6561279296875, 0.7171630859375, 0.67138671875, 0.640869140625, 0.6256103515625, 0.6103515625, 0.6256103515625, 0.6561279296875, 0.67138671875, 0.6103515625, 0.6256103515625, 0.67138671875, 0.640869140625, 0.732421875, 0.701904296875, 0.6866455078125, 0.6866455078125, 0.6561279296875, 0.6103515625, 0.6866455078125, 0.6561279296875, 0.67138671875, 0.6561279296875, 0.6561279296875, 0.67138671875, 0.701904296875, 0.6561279296875, 0.6256103515625, 0.6561279296875, 0.6561279296875, 0.7171630859375, 0.640869140625, 0.640869140625, 0.6561279296875, 0.640869140625, 0.640869140625, 0.6561279296875, 0.640869140625, 0.701904296875, 0.6866455078125, 0.701904296875, 0.67138671875, 0.640869140625, 0.6866455078125, 0.701904296875, 0.6866455078125]

In [45]:
gx = [-1.3275146484375, -1.3885498046875, -1.3275146484375, -1.3427734375, -1.373291015625, -1.2969970703125, -1.3275146484375, -1.312255859375, -1.3427734375, -1.312255859375, -1.3275146484375, -1.373291015625, -1.312255859375, -1.3580322265625, -1.312255859375, -1.3275146484375, -1.3427734375, -1.2969970703125, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.312255859375, -1.3427734375, -1.312255859375, -1.28173828125, -1.312255859375, -1.312255859375, -1.3275146484375, -1.2969970703125, -1.3275146484375, -1.2664794921875, -1.3427734375, -1.2969970703125, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.3427734375, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.373291015625, -1.2969970703125, -1.312255859375, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.373291015625, -1.3275146484375, -1.373291015625, -1.3427734375, -1.3427734375, -1.312255859375, -1.2969970703125, -1.3427734375, -1.373291015625, -1.3275146484375, -1.373291015625, -1.312255859375, -1.373291015625, -1.40380859375, -1.3580322265625, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.3885498046875, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.3275146484375, -1.373291015625, -1.3275146484375, -1.312255859375, -1.2969970703125, -1.3275146484375, -1.3427734375, -1.373291015625, -1.3427734375, -1.3275146484375, -1.312255859375, -1.3580322265625, -1.3580322265625, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3275146484375, -1.3427734375, -1.3427734375, -1.3885498046875, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.2969970703125, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.3427734375, -1.312255859375, -1.3580322265625, -1.3580322265625, -1.3427734375, -1.373291015625, -1.373291015625, -1.3885498046875, -1.312255859375, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.373291015625, -1.3885498046875, -1.2969970703125, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.373291015625, -1.3275146484375, -1.373291015625, -1.3427734375, -1.373291015625, -1.3427734375, -1.2969970703125, -1.3427734375, -1.3580322265625, -1.312255859375, -1.40380859375, -1.40380859375, -1.373291015625, -1.2969970703125, -1.373291015625, -1.3580322265625, -1.2969970703125, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.312255859375, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3427734375, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.312255859375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3885498046875, -1.3275146484375, -1.312255859375, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.373291015625, -1.3275146484375, -1.312255859375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3885498046875, -1.40380859375, -1.3885498046875, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.373291015625, -1.312255859375, -1.3427734375, -1.3427734375, -1.373291015625, -1.3275146484375, -1.312255859375, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.3580322265625, -1.3275146484375, -1.3580322265625, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3427734375, -1.3427734375, -1.373291015625, -1.3885498046875, -1.3885498046875, -1.373291015625, -1.3885498046875, -1.3580322265625, -1.40380859375, -1.3580322265625, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3427734375, -1.373291015625, -1.3427734375, -1.312255859375, -1.3427734375, -1.2969970703125, -1.2969970703125, -1.3427734375, -1.3275146484375, -1.3580322265625, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.312255859375, -1.2664794921875, -1.3275146484375, -1.3427734375, -1.373291015625, -1.3580322265625, -1.373291015625, -1.3580322265625, -1.3275146484375, -1.312255859375, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.312255859375, -1.3580322265625, -1.40380859375, -1.3885498046875, -1.3275146484375, -1.3275146484375, -1.312255859375, -1.3580322265625, -1.373291015625, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.312255859375, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.373291015625, -1.312255859375, -1.28173828125, -1.3275146484375, -1.2969970703125, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.312255859375, -1.3580322265625, -1.3427734375, -1.312255859375, -1.2969970703125, -1.3275146484375, -1.373291015625, -1.3580322265625, -1.3580322265625, -1.373291015625, -1.312255859375, -1.3427734375, -1.3427734375, -1.3427734375, -1.3427734375, -1.3427734375, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3427734375, -1.3427734375, -1.3580322265625, -1.373291015625, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.312255859375, -1.28173828125, -1.3275146484375, -1.312255859375, -1.2969970703125, -1.3580322265625, -1.3275146484375, -1.3275146484375, -1.3275146484375, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.373291015625, -1.3580322265625, -1.373291015625, -1.3885498046875, -1.373291015625, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.373291015625, -1.3275146484375, -1.312255859375, -1.373291015625, -1.373291015625, -1.312255859375, -1.2969970703125, -1.312255859375, -1.312255859375, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3275146484375, -1.3580322265625, -1.3580322265625, -1.3275146484375, -1.312255859375, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.3427734375, -1.373291015625, -1.2969970703125, -1.3427734375, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.373291015625, -1.373291015625, -1.3580322265625, -1.3580322265625, -1.312255859375, -1.312255859375, -1.3275146484375, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3275146484375, -1.3580322265625, -1.4190673828125, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.373291015625, -1.3427734375, -1.312255859375, -1.312255859375, -1.312255859375, -1.312255859375, -1.3427734375, -1.3427734375, -1.3275146484375, -1.3427734375, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.3275146484375, -1.312255859375, -1.3275146484375, -1.3275146484375, -1.312255859375, -1.312255859375, -1.2969970703125, -1.3427734375, -1.3427734375, -1.3885498046875, -1.3885498046875, -1.3885498046875, -1.3275146484375, -1.28173828125, -1.3427734375, -1.373291015625, -1.373291015625, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.3427734375, -1.3427734375, -1.3580322265625, -1.373291015625, -1.373291015625, -1.3275146484375, -1.3580322265625, -1.3275146484375, -1.2969970703125, -1.3275146484375, -1.312255859375, -1.28173828125, -1.2969970703125, -1.312255859375, -1.312255859375, -1.3275146484375, -1.3275146484375, -1.3275146484375, -1.3275146484375, -1.3275146484375, -1.2969970703125, -1.3275146484375, -1.312255859375, -1.3275146484375, -1.373291015625, -1.3580322265625, -1.3275146484375, -1.3275146484375, -1.3275146484375, -1.312255859375, -1.3427734375, -1.373291015625, -1.3427734375, -1.312255859375, -1.312255859375, -1.3427734375, -1.2969970703125, -1.3275146484375, -1.3427734375, -1.3580322265625, -1.3580322265625, -1.3427734375, -1.3885498046875, -1.3580322265625, -1.3427734375, -1.40380859375, -1.373291015625, -1.3275146484375, -1.3427734375, -1.373291015625, -1.373291015625, -1.3275146484375, -1.3580322265625, -1.373291015625, -1.3885498046875, -1.2969970703125, -1.2969970703125, -1.28173828125, -1.373291015625, -1.3580322265625, -1.3427734375, -1.3427734375, -1.3275146484375, -1.373291015625, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.312255859375, -1.3580322265625, -1.373291015625, -1.3427734375, -1.3275146484375, -1.3275146484375, -1.3580322265625, -1.3275146484375, -1.3275146484375, -1.3427734375, -1.373291015625, -1.2664794921875, -1.2969970703125, -1.3275146484375, -1.2969970703125, -1.3275146484375, -1.312255859375, -1.312255859375, -1.373291015625, -1.3427734375, -1.312255859375]

In [46]:
test_data = np.array([ax, ay, az, gx, gy, gz]).T

In [47]:
test_data

array([[-0.07409668,  0.5012207 ,  0.90222168, -1.32751465,  0.62561035,
         2.21252441],
       [-0.07373047,  0.50244141,  0.90124512, -1.3885498 ,  0.61035156,
         2.25830078],
       [-0.07336426,  0.50195312,  0.8996582 , -1.32751465,  0.62561035,
         2.24304199],
       ...,
       [-0.07434082,  0.50292969,  0.90185547, -1.37329102,  0.68664551,
         2.18200684],
       [-0.07458496,  0.50158691,  0.90148926, -1.34277344,  0.7019043 ,
         2.18200684],
       [-0.07434082,  0.50219727,  0.90112305, -1.31225586,  0.68664551,
         2.18200684]])

In [48]:
is_correct_accel_data(test_data)

1

In [49]:
mg = [np.linalg.norm(x[:3]) for x in test_data]
sd= np.std(mg)

In [79]:
ax, ay, az, gx, gy, gz = [*pdf['data'].iloc[0][0]], [*pdf['data'].iloc[0][1]], [*pdf['data'].iloc[0][2]], [*pdf['data'].iloc[0][3]], [*pdf['data'].iloc[0][4]], [*pdf['data'].iloc[0][5]]

In [81]:
td2 = np.array([ax, ay,az,gx,gy,gz])

In [96]:
len(ax)

477

In [97]:
td3 = np.array([*ax, *ay, *az, *gx, *gy, *gz])

In [98]:
td3

array([-0.07421875, -0.07312012, -0.07409668, ...,  2.18200684,
        2.16674805,  2.19726562])

In [99]:
td4 = td3.reshape(-1, 477)

In [103]:
td4.T

array([[-0.07421875,  0.50219727,  0.90161133, ...,  0.59509277,
         2.12097168,  2.12097168],
       [-0.07312012,  0.5020752 ,  0.9029541 , ...,  0.65612793,
         2.12097168,  2.12097168],
       [-0.07409668,  0.50134277,  0.9029541 , ...,  0.61035156,
         2.16674805,  2.16674805],
       ...,
       [-0.07275391,  0.50183105,  0.90112305, ...,  0.65612793,
         2.18200684,  2.18200684],
       [-0.07263184,  0.5012207 ,  0.90222168, ...,  0.68664551,
         2.16674805,  2.16674805],
       [-0.07312012,  0.50158691,  0.90075684, ...,  0.68664551,
         2.19726562,  2.19726562]])

In [155]:
ttest = np.array([[-0.234130859375, 0.093017578125, 0.9979248046875],
 [-0.2337646484375, 0.092041015625, 0.997314453125],
 [-0.2333984375, 0.0914306640625, 0.9984130859375],
 [-0.234375, 0.0914306640625, 0.9981689453125],
 [-0.23291015625, 0.0909423828125, 0.998046875],
 [-0.23388671875, 0.0924072265625, 0.99755859375],
 [-0.2340087890625, 0.0921630859375, 0.99853515625],
 [-0.2347412109375, 0.0919189453125, 0.9971923828125],
 [-0.233642578125, 0.0904541015625, 0.9964599609375],
 [-0.2335205078125, 0.091552734375, 0.9976806640625],
 [-0.2335205078125, 0.091796875, 0.997802734375],
 [-0.234375, 0.091552734375, 0.9996337890625],
 [-0.234375, 0.0919189453125, 0.9979248046875],
 [-0.2353515625, 0.0919189453125, 0.9981689453125],
 [-0.234130859375, 0.091064453125, 0.999267578125],
 [-0.23291015625, 0.0919189453125, 0.998291015625],
 [-0.2335205078125, 0.0919189453125, 0.9970703125],
 [-0.2333984375, 0.0909423828125, 0.99609375],
 [-0.234130859375, 0.0916748046875, 0.9981689453125],
 [-0.233154296875, 0.0909423828125, 0.9989013671875],
 [-0.234375, 0.0911865234375, 0.997802734375],
 [-0.233154296875, 0.092041015625, 0.9970703125],
 [-0.23388671875, 0.091064453125, 0.9969482421875],
 [-0.2342529296875, 0.0911865234375, 0.998291015625],
 [-0.2342529296875, 0.0911865234375, 0.998291015625],
 [-0.2327880859375, 0.0916748046875, 0.9984130859375],
 [-0.23388671875, 0.0914306640625, 0.9990234375],
 [-0.2342529296875, 0.092041015625, 0.99853515625],
 [-0.234375, 0.0909423828125, 0.998291015625],
 [-0.2332763671875, 0.0924072265625, 0.997802734375],
 [-0.2333984375, 0.091552734375, 0.9974365234375],
 [-0.23388671875, 0.0914306640625, 0.997314453125],
 [-0.2330322265625, 0.0921630859375, 1.0001220703125],
 [-0.2337646484375, 0.09130859375, 0.997802734375],
 [-0.233642578125, 0.091552734375, 0.9984130859375],
 [-0.2340087890625, 0.0919189453125, 0.99853515625],
 [-0.233154296875, 0.0919189453125, 0.999755859375],
 [-0.233642578125, 0.090576171875, 0.997802734375],
 [-0.233154296875, 0.091796875, 0.9984130859375],
 [-0.2333984375, 0.09130859375, 0.9964599609375],
 [-0.2335205078125, 0.092041015625, 0.9964599609375],
 [-0.2337646484375, 0.09130859375, 0.997802734375],
 [-0.2337646484375, 0.0911865234375, 0.9981689453125],
 [-0.2340087890625, 0.09130859375, 0.99755859375],
 [-0.23388671875, 0.09130859375, 0.996826171875],
 [-0.2332763671875, 0.091064453125, 0.9981689453125],
 [-0.23388671875, 0.091552734375, 0.9984130859375],
 [-0.232666015625, 0.09130859375, 0.9974365234375],
 [-0.2340087890625, 0.0904541015625, 0.9964599609375],
 [-0.2332763671875, 0.0919189453125, 0.99560546875],
 [-0.2337646484375, 0.0904541015625, 0.998291015625],
 [-0.2337646484375, 0.0909423828125, 0.9976806640625],
 [-0.2340087890625, 0.09130859375, 0.9967041015625],
 [-0.2333984375, 0.0909423828125, 0.99658203125],
 [-0.234375, 0.0908203125, 0.9964599609375],
 [-0.2340087890625, 0.091552734375, 0.9970703125],
 [-0.23291015625, 0.09130859375, 0.9991455078125],
 [-0.233642578125, 0.0914306640625, 0.99853515625],
 [-0.2342529296875, 0.0909423828125, 0.998291015625],
 [-0.2342529296875, 0.092041015625, 0.9991455078125],
 [-0.2337646484375, 0.0906982421875, 0.998291015625],
 [-0.2333984375, 0.091064453125, 0.99853515625],
 [-0.2332763671875, 0.0904541015625, 0.9970703125],
 [-0.2333984375, 0.091552734375, 0.9971923828125],
 [-0.2335205078125, 0.092041015625, 0.9974365234375],
 [-0.2333984375, 0.0914306640625, 0.9991455078125],
 [-0.233642578125, 0.0908203125, 0.995361328125],
 [-0.2335205078125, 0.0928955078125, 0.9971923828125],
 [-0.2340087890625, 0.0919189453125, 0.998291015625],
 [-0.233642578125, 0.0911865234375, 0.998291015625],
 [-0.233642578125, 0.092041015625, 0.9974365234375],
 [-0.2327880859375, 0.091796875, 0.99755859375],
 [-0.23291015625, 0.0916748046875, 0.99658203125],
 [-0.2333984375, 0.0911865234375, 0.9976806640625],
 [-0.2332763671875, 0.0914306640625, 0.9971923828125],
 [-0.2327880859375, 0.0921630859375, 0.9964599609375],
 [-0.233642578125, 0.092041015625, 0.99609375],
 [-0.2330322265625, 0.0911865234375, 0.9970703125],
 [-0.233642578125, 0.09130859375, 0.997802734375],
 [-0.2335205078125, 0.0924072265625, 0.998291015625],
 [-0.2337646484375, 0.0919189453125, 0.998046875],
 [-0.2333984375, 0.0914306640625, 0.9979248046875],
 [-0.23291015625, 0.091552734375, 0.998046875],
 [-0.233642578125, 0.092041015625, 0.997314453125],
 [-0.234130859375, 0.092041015625, 0.9990234375],
 [-0.23388671875, 0.0911865234375, 0.998291015625],
 [-0.2335205078125, 0.091552734375, 0.9981689453125],
 [-0.2337646484375, 0.0914306640625, 0.9969482421875],
 [-0.234619140625, 0.091064453125, 0.99755859375],
 [-0.234130859375, 0.0916748046875, 0.997802734375],
 [-0.2342529296875, 0.09130859375, 0.9990234375],
 [-0.2340087890625, 0.0916748046875, 0.9976806640625],
 [-0.2333984375, 0.0911865234375, 0.9976806640625],
 [-0.2333984375, 0.091552734375, 0.9979248046875],
 [-0.2337646484375, 0.092041015625, 0.998046875],
 [-0.2322998046875, 0.089599609375, 0.9954833984375],
 [-0.234130859375, 0.0911865234375, 0.9971923828125],
 [-0.2342529296875, 0.0909423828125, 0.99853515625],
 [-0.234375, 0.09228515625, 0.9991455078125],
 [-0.2332763671875, 0.0914306640625, 0.99951171875],
 [-0.23388671875, 0.0906982421875, 0.9976806640625],
 [-0.233154296875, 0.09228515625, 0.9974365234375],
 [-0.234130859375, 0.091552734375, 0.9984130859375],
 [-0.2330322265625, 0.0914306640625, 0.9974365234375],
 [-0.2335205078125, 0.0919189453125, 0.998046875],
 [-0.2337646484375, 0.0916748046875, 0.998046875],
 [-0.23486328125, 0.0914306640625, 0.997802734375],
 [-0.234375, 0.0916748046875, 0.998291015625],
 [-0.2340087890625, 0.092041015625, 0.9962158203125],
 [-0.2337646484375, 0.091796875, 0.998291015625],
 [-0.2337646484375, 0.09130859375, 0.9974365234375],
 [-0.2335205078125, 0.0921630859375, 0.9957275390625],
 [-0.23291015625, 0.0916748046875, 0.99755859375],
 [-0.2335205078125, 0.0919189453125, 0.996826171875],
 [-0.2335205078125, 0.0924072265625, 0.997802734375],
 [-0.233642578125, 0.091552734375, 0.997314453125],
 [-0.2332763671875, 0.092041015625, 0.9976806640625],
 [-0.233642578125, 0.0908203125, 0.998046875],
 [-0.2330322265625, 0.0921630859375, 0.9971923828125],
 [-0.234130859375, 0.09130859375, 0.9996337890625],
 [-0.233154296875, 0.0916748046875, 0.9970703125],
 [-0.2337646484375, 0.0916748046875, 0.997314453125],
 [-0.2335205078125, 0.09228515625, 0.999267578125],
 [-0.234619140625, 0.091796875, 0.9986572265625],
 [-0.2340087890625, 0.09130859375, 0.9979248046875],
 [-0.2335205078125, 0.0914306640625, 0.9962158203125],
 [-0.234130859375, 0.0921630859375, 0.998779296875],
 [-0.2340087890625, 0.09228515625, 0.996826171875],
 [-0.2340087890625, 0.09130859375, 0.9974365234375],
 [-0.2337646484375, 0.091552734375, 0.9981689453125],
 [-0.233154296875, 0.091796875, 0.9969482421875],
 [-0.2325439453125, 0.0916748046875, 0.9981689453125],
 [-0.2333984375, 0.091552734375, 0.998291015625],
 [-0.234130859375, 0.091796875, 0.9979248046875],
 [-0.2332763671875, 0.0911865234375, 0.9969482421875],
 [-0.2340087890625, 0.0916748046875, 0.9974365234375],
 [-0.23388671875, 0.092529296875, 0.9979248046875],
 [-0.2332763671875, 0.0919189453125, 0.9967041015625],
 [-0.2337646484375, 0.0908203125, 0.9976806640625],
 [-0.2333984375, 0.0914306640625, 0.997802734375],
 [-0.2325439453125, 0.092041015625, 0.9970703125],
 [-0.2333984375, 0.092041015625, 0.9984130859375],
 [-0.2327880859375, 0.091552734375, 0.9984130859375],
 [-0.2337646484375, 0.0914306640625, 0.999267578125],
 [-0.2337646484375, 0.0909423828125, 0.9991455078125],
 [-0.2332763671875, 0.091064453125, 0.99755859375],
 [-0.2333984375, 0.092529296875, 0.9967041015625],
 [-0.2335205078125, 0.09130859375, 0.9979248046875],
 [-0.233154296875, 0.0904541015625, 0.9969482421875],
 [-0.233642578125, 0.091064453125, 0.997802734375],
 [-0.233642578125, 0.091064453125, 0.9990234375],
 [-0.2332763671875, 0.0916748046875, 0.9979248046875],
 [-0.23388671875, 0.0899658203125, 0.9976806640625],
 [-0.2332763671875, 0.09130859375, 0.997802734375],
 [-0.233642578125, 0.09228515625, 0.9990234375],
 [-0.2332763671875, 0.0926513671875, 0.9981689453125],
 [-0.2330322265625, 0.0914306640625, 0.9981689453125],
 [-0.233642578125, 0.0927734375, 0.9976806640625],
 [-0.23388671875, 0.092529296875, 0.9970703125],
 [-0.2332763671875, 0.09228515625, 0.996826171875],
 [-0.2342529296875, 0.091064453125, 0.998046875],
 [-0.233642578125, 0.0906982421875, 0.9989013671875],
 [-0.2342529296875, 0.0916748046875, 0.99755859375],
 [-0.2332763671875, 0.0916748046875, 0.997314453125],
 [-0.2337646484375, 0.0906982421875, 0.998291015625],
 [-0.2335205078125, 0.090576171875, 0.9969482421875],
 [-0.2340087890625, 0.091796875, 0.9971923828125],
 [-0.234619140625, 0.09130859375, 0.9974365234375],
 [-0.2332763671875, 0.0911865234375, 0.998779296875],
 [-0.23291015625, 0.091552734375, 0.997802734375],
 [-0.234619140625, 0.091064453125, 0.9979248046875],
 [-0.2333984375, 0.0916748046875, 0.99462890625],
 [-0.2333984375, 0.09228515625, 0.9974365234375],
 [-0.233154296875, 0.0916748046875, 0.9984130859375],
 [-0.23388671875, 0.0911865234375, 0.9970703125],
 [-0.233154296875, 0.0919189453125, 0.997802734375],
 [-0.2335205078125, 0.091796875, 0.99853515625],
 [-0.2330322265625, 0.0914306640625, 0.9979248046875],
 [-0.233642578125, 0.0909423828125, 0.99853515625],
 [-0.2332763671875, 0.0916748046875, 0.997314453125],
 [-0.2335205078125, 0.0919189453125, 0.9976806640625],
 [-0.2340087890625, 0.0908203125, 0.997314453125],
 [-0.233642578125, 0.0911865234375, 0.998046875],
 [-0.234130859375, 0.0908203125, 0.9970703125],
 [-0.23388671875, 0.0911865234375, 0.99609375],
 [-0.233642578125, 0.0926513671875, 0.99853515625],
 [-0.23486328125, 0.0914306640625, 0.9984130859375],
 [-0.233154296875, 0.0914306640625, 0.999267578125],
 [-0.234130859375, 0.091064453125, 0.997802734375],
 [-0.234130859375, 0.091796875, 0.9986572265625],
 [-0.2337646484375, 0.0916748046875, 0.9976806640625],
 [-0.2342529296875, 0.091064453125, 0.997802734375],
 [-0.2332763671875, 0.0904541015625, 0.9974365234375],
 [-0.2340087890625, 0.0911865234375, 0.9979248046875],
 [-0.2349853515625, 0.091552734375, 0.9970703125],
 [-0.234130859375, 0.091064453125, 0.995849609375],
 [-0.233154296875, 0.0916748046875, 0.9969482421875],
 [-0.2337646484375, 0.0919189453125, 0.9945068359375],
 [-0.2335205078125, 0.091796875, 0.9974365234375],
 [-0.2335205078125, 0.0921630859375, 0.99755859375],
 [-0.2333984375, 0.0909423828125, 0.9970703125],
 [-0.233642578125, 0.091064453125, 0.99853515625],
 [-0.2332763671875, 0.0911865234375, 0.996826171875],
 [-0.2330322265625, 0.0909423828125, 0.997802734375],
 [-0.2337646484375, 0.0916748046875, 0.997802734375],
 [-0.2335205078125, 0.09130859375, 0.9979248046875],
 [-0.2337646484375, 0.091552734375, 0.9981689453125],
 [-0.234130859375, 0.0914306640625, 0.9998779296875],
 [-0.23291015625, 0.092041015625, 0.99755859375],
 [-0.2337646484375, 0.09228515625, 0.996826171875],
 [-0.23388671875, 0.0926513671875, 0.9976806640625],
 [-0.2332763671875, 0.091552734375, 0.9979248046875],
 [-0.234130859375, 0.0916748046875, 0.99755859375],
 [-0.2337646484375, 0.09130859375, 0.9962158203125],
 [-0.2340087890625, 0.091064453125, 0.9974365234375],
 [-0.2320556640625, 0.0914306640625, 0.997314453125],
 [-0.2340087890625, 0.0919189453125, 0.998046875],
 [-0.2344970703125, 0.0904541015625, 0.998046875],
 [-0.23486328125, 0.091064453125, 0.997802734375],
 [-0.2347412109375, 0.0911865234375, 0.9971923828125],
 [-0.2335205078125, 0.0914306640625, 0.997314453125],
 [-0.234375, 0.0914306640625, 0.997802734375],
 [-0.2347412109375, 0.091552734375, 0.9984130859375],
 [-0.2342529296875, 0.09228515625, 0.9986572265625],
 [-0.234130859375, 0.0924072265625, 0.997802734375],
 [-0.2332763671875, 0.0916748046875, 0.9959716796875],
 [-0.2340087890625, 0.09130859375, 0.9986572265625],
 [-0.234375, 0.0921630859375, 0.9974365234375],
 [-0.23486328125, 0.09228515625, 0.9971923828125],
 [-0.233642578125, 0.0914306640625, 0.9984130859375],
 [-0.2337646484375, 0.0926513671875, 0.996826171875],
 [-0.2337646484375, 0.091796875, 0.997802734375],
 [-0.2337646484375, 0.0909423828125, 0.997314453125],
 [-0.234619140625, 0.0924072265625, 0.9967041015625],
 [-0.234375, 0.091796875, 0.9991455078125],
 [-0.2335205078125, 0.091552734375, 0.9971923828125],
 [-0.2337646484375, 0.0926513671875, 0.99755859375],
 [-0.2332763671875, 0.092041015625, 0.9976806640625],
 [-0.2335205078125, 0.0914306640625, 0.997802734375],
 [-0.2325439453125, 0.09130859375, 0.99560546875],
 [-0.2340087890625, 0.09130859375, 0.99853515625],
 [-0.2342529296875, 0.09228515625, 0.998291015625],
 [-0.232666015625, 0.091552734375, 0.99853515625],
 [-0.2337646484375, 0.0916748046875, 0.997802734375],
 [-0.2332763671875, 0.0919189453125, 0.9991455078125],
 [-0.2347412109375, 0.0914306640625, 0.99755859375],
 [-0.2333984375, 0.091796875, 0.9979248046875],
 [-0.234130859375, 0.09130859375, 0.999267578125],
 [-0.234619140625, 0.09130859375, 0.997314453125],
 [-0.2337646484375, 0.0916748046875, 0.9971923828125],
 [-0.234130859375, 0.0914306640625, 0.9959716796875],
 [-0.233154296875, 0.091796875, 0.998291015625],
 [-0.2332763671875, 0.0928955078125, 0.998291015625],
 [-0.233154296875, 0.0916748046875, 0.9989013671875],
 [-0.2337646484375, 0.0919189453125, 0.9970703125],
 [-0.2337646484375, 0.0924072265625, 0.9974365234375],
 [-0.2340087890625, 0.091552734375, 0.998779296875],
 [-0.2330322265625, 0.0916748046875, 0.998291015625],
 [-0.23291015625, 0.0906982421875, 0.99755859375],
 [-0.2332763671875, 0.091552734375, 0.9976806640625],
 [-0.23388671875, 0.0916748046875, 0.995849609375],
 [-0.23388671875, 0.0916748046875, 0.99951171875],
 [-0.233642578125, 0.0914306640625, 1.001220703125],
 [-0.2337646484375, 0.0908203125, 0.99951171875],
 [-0.233642578125, 0.0911865234375, 0.998291015625],
 [-0.2332763671875, 0.092041015625, 0.9979248046875],
 [-0.234130859375, 0.09130859375, 0.99755859375],
 [-0.23388671875, 0.0924072265625, 0.9974365234375],
 [-0.2335205078125, 0.091552734375, 0.997314453125],
 [-0.2327880859375, 0.091552734375, 0.998779296875],
 [-0.233154296875, 0.0921630859375, 0.9986572265625],
 [-0.2335205078125, 0.091064453125, 0.9981689453125],
 [-0.2337646484375, 0.092041015625, 0.99658203125],
 [-0.2337646484375, 0.09033203125, 0.9991455078125],
 [-0.2327880859375, 0.0906982421875, 0.9990234375],
 [-0.233642578125, 0.09228515625, 0.9970703125],
 [-0.233154296875, 0.0908203125, 0.998291015625],
 [-0.23388671875, 0.0908203125, 0.997802734375],
 [-0.2340087890625, 0.0906982421875, 0.9957275390625],
 [-0.2342529296875, 0.0909423828125, 0.9979248046875],
 [-0.23291015625, 0.09228515625, 0.9998779296875],
 [-0.232666015625, 0.091796875, 0.9984130859375],
 [-0.232666015625, 0.0921630859375, 0.9984130859375],
 [-0.232666015625, 0.091796875, 0.9974365234375],
 [-0.234619140625, 0.092041015625, 0.999267578125],
 [-0.234130859375, 0.0916748046875, 0.9974365234375],
 [-0.2337646484375, 0.0904541015625, 0.9969482421875],
 [-0.2335205078125, 0.09033203125, 0.99755859375],
 [-0.2342529296875, 0.091796875, 0.9991455078125],
 [-0.234130859375, 0.0911865234375, 0.9979248046875],
 [-0.23291015625, 0.090576171875, 0.997314453125],
 [-0.23388671875, 0.0914306640625, 0.9990234375],
 [-0.2344970703125, 0.0906982421875, 0.998291015625],
 [-0.2335205078125, 0.0924072265625, 0.9993896484375],
 [-0.233642578125, 0.09130859375, 0.9981689453125],
 [-0.233154296875, 0.0916748046875, 0.9974365234375],
 [-0.2344970703125, 0.0909423828125, 0.99658203125],
 [-0.2333984375, 0.091796875, 0.99853515625],
 [-0.2333984375, 0.092041015625, 0.996337890625],
 [-0.2352294921875, 0.092041015625, 0.998046875],
 [-0.23388671875, 0.091796875, 0.9976806640625],
 [-0.233154296875, 0.0919189453125, 0.996826171875],
 [-0.2340087890625, 0.091064453125, 0.998291015625],
 [-0.2332763671875, 0.0914306640625, 0.9970703125],
 [-0.233154296875, 0.0904541015625, 0.996826171875],
 [-0.23388671875, 0.091796875, 0.9970703125],
 [-0.233154296875, 0.0916748046875, 0.9974365234375],
 [-0.233642578125, 0.091796875, 0.9971923828125],
 [-0.234130859375, 0.091552734375, 0.998046875],
 [-0.2337646484375, 0.092041015625, 0.99951171875],
 [-0.2333984375, 0.0926513671875, 0.9986572265625],
 [-0.2337646484375, 0.092041015625, 0.998291015625],
 [-0.233154296875, 0.0921630859375, 0.998779296875],
 [-0.233642578125, 0.0916748046875, 0.9976806640625],
 [-0.2332763671875, 0.0919189453125, 0.9970703125],
 [-0.2330322265625, 0.0919189453125, 0.9976806640625],
 [-0.2332763671875, 0.092041015625, 0.9969482421875],
 [-0.2344970703125, 0.09228515625, 0.9979248046875],
 [-0.2330322265625, 0.0911865234375, 0.9974365234375],
 [-0.23486328125, 0.0924072265625, 0.9981689453125],
 [-0.234375, 0.091552734375, 0.99755859375],
 [-0.233642578125, 0.0928955078125, 0.9986572265625],
 [-0.233642578125, 0.0928955078125, 0.9986572265625],
 [-0.2325439453125, 0.0916748046875, 0.999267578125],
 [-0.23291015625, 0.090576171875, 0.9971923828125],
 [-0.2335205078125, 0.0914306640625, 0.99755859375],
 [-0.233642578125, 0.0919189453125, 0.9976806640625],
 [-0.234619140625, 0.0919189453125, 0.998291015625],
 [-0.2349853515625, 0.0916748046875, 0.996826171875],
 [-0.234619140625, 0.0909423828125, 0.9981689453125],
 [-0.2340087890625, 0.091796875, 0.9969482421875],
 [-0.234375, 0.0909423828125, 0.9976806640625],
 [-0.2340087890625, 0.091064453125, 0.996826171875],
 [-0.23388671875, 0.092041015625, 0.99755859375],
 [-0.234130859375, 0.0911865234375, 0.9981689453125],
 [-0.2340087890625, 0.0921630859375, 0.997314453125],
 [-0.234375, 0.092041015625, 0.998291015625],
 [-0.233642578125, 0.0904541015625, 0.9979248046875],
 [-0.233154296875, 0.0914306640625, 0.9984130859375],
 [-0.233642578125, 0.0902099609375, 0.9989013671875],
 [-0.234130859375, 0.0914306640625, 0.99755859375],
 [-0.2337646484375, 0.0919189453125, 0.9979248046875],
 [-0.23388671875, 0.0911865234375, 0.998046875],
 [-0.233154296875, 0.0919189453125, 0.9969482421875],
 [-0.2340087890625, 0.0921630859375, 0.9971923828125],
 [-0.23388671875, 0.0919189453125, 0.995849609375],
 [-0.2342529296875, 0.09130859375, 0.998046875],
 [-0.2335205078125, 0.0914306640625, 0.99853515625],
 [-0.2335205078125, 0.090087890625, 0.9967041015625],
 [-0.2347412109375, 0.09130859375, 0.998046875],
 [-0.2342529296875, 0.0909423828125, 0.998291015625],
 [-0.2332763671875, 0.0909423828125, 0.9971923828125],
 [-0.233642578125, 0.0921630859375, 0.9957275390625],
 [-0.2335205078125, 0.0909423828125, 0.9974365234375],
 [-0.234130859375, 0.0908203125, 0.9979248046875],
 [-0.233154296875, 0.0919189453125, 0.9976806640625],
 [-0.234375, 0.091064453125, 0.997314453125],
 [-0.2340087890625, 0.0914306640625, 0.9974365234375],
 [-0.2335205078125, 0.091796875, 0.998779296875],
 [-0.2335205078125, 0.0914306640625, 0.99951171875],
 [-0.2332763671875, 0.0916748046875, 0.996826171875],
 [-0.233154296875, 0.0919189453125, 0.998779296875],
 [-0.2340087890625, 0.0899658203125, 0.998046875],
 [-0.23486328125, 0.091064453125, 0.9986572265625],
 [-0.23388671875, 0.092041015625, 0.998291015625],
 [-0.233154296875, 0.0927734375, 0.99658203125],
 [-0.2337646484375, 0.0904541015625, 0.997802734375],
 [-0.2333984375, 0.0909423828125, 0.998046875],
 [-0.234619140625, 0.0911865234375, 0.99755859375],
 [-0.2340087890625, 0.09130859375, 0.9981689453125],
 [-0.2337646484375, 0.0916748046875, 0.997314453125],
 [-0.2342529296875, 0.0916748046875, 0.9979248046875],
 [-0.234130859375, 0.0906982421875, 0.9974365234375],
 [-0.23486328125, 0.091796875, 0.9969482421875],
 [-0.233642578125, 0.09228515625, 0.9981689453125],
 [-0.2344970703125, 0.092529296875, 0.998779296875],
 [-0.2333984375, 0.09228515625, 0.9981689453125],
 [-0.2330322265625, 0.0916748046875, 0.9957275390625],
 [-0.2347412109375, 0.0919189453125, 0.998046875],
 [-0.234130859375, 0.0914306640625, 0.9970703125],
 [-0.2337646484375, 0.0921630859375, 0.9981689453125],
 [-0.2333984375, 0.091796875, 0.9974365234375],
 [-0.2327880859375, 0.0911865234375, 0.998046875],
 [-0.2340087890625, 0.091796875, 0.99755859375],
 [-0.233642578125, 0.091796875, 0.9984130859375],
 [-0.234619140625, 0.092041015625, 0.9986572265625],
 [-0.235595703125, 0.091064453125, 0.9976806640625],
 [-0.234619140625, 0.0914306640625, 0.9967041015625],
 [-0.2342529296875, 0.091552734375, 0.997314453125],
 [-0.2344970703125, 0.0916748046875, 0.998291015625],
 [-0.23388671875, 0.09130859375, 0.9970703125],
 [-0.2340087890625, 0.092041015625, 0.9989013671875],
 [-0.23388671875, 0.091552734375, 0.99609375],
 [-0.2333984375, 0.0909423828125, 0.997802734375],
 [-0.2337646484375, 0.0914306640625, 0.998291015625],
 [-0.2335205078125, 0.09130859375, 0.998779296875],
 [-0.2333984375, 0.092041015625, 0.99560546875],
 [-0.23388671875, 0.0906982421875, 0.9979248046875],
 [-0.234130859375, 0.0916748046875, 0.9998779296875],
 [-0.2333984375, 0.091064453125, 0.99755859375],
 [-0.232666015625, 0.0908203125, 0.9962158203125],
 [-0.2342529296875, 0.09130859375, 0.996826171875],
 [-0.234130859375, 0.091064453125, 0.9989013671875],
 [-0.2347412109375, 0.09130859375, 0.99755859375],
 [-0.2342529296875, 0.0924072265625, 0.9954833984375],
 [-0.2340087890625, 0.092041015625, 0.998291015625],
 [-0.2340087890625, 0.09130859375, 0.9964599609375],
 [-0.2330322265625, 0.0911865234375, 0.9962158203125],
 [-0.2332763671875, 0.0911865234375, 0.9967041015625],
 [-0.234619140625, 0.091796875, 0.9974365234375],
 [-0.234375, 0.0926513671875, 0.9976806640625],
 [-0.23291015625, 0.0916748046875, 0.998291015625],
 [-0.2337646484375, 0.0914306640625, 0.998291015625],
 [-0.2337646484375, 0.0919189453125, 0.996337890625],
 [-0.2340087890625, 0.091552734375, 0.9969482421875],
 [-0.234375, 0.0916748046875, 0.9989013671875],
 [-0.2330322265625, 0.092041015625, 0.99658203125],
 [-0.233642578125, 0.0921630859375, 0.9979248046875],
 [-0.2332763671875, 0.091796875, 0.99755859375],
 [-0.2333984375, 0.0906982421875, 0.9976806640625],
 [-0.235107421875, 0.0921630859375, 0.9984130859375],
 [-0.2349853515625, 0.0921630859375, 0.9976806640625],
 [-0.2337646484375, 0.0919189453125, 0.9974365234375],
 [-0.23388671875, 0.091552734375, 0.9981689453125],
 [-0.234130859375, 0.0914306640625, 0.998291015625],
 [-0.2337646484375, 0.09228515625, 0.9970703125],
 [-0.2340087890625, 0.091796875, 0.99755859375],
 [-0.2342529296875, 0.0916748046875, 0.9979248046875],
 [-0.2337646484375, 0.09228515625, 0.9971923828125],
 [-0.2342529296875, 0.091796875, 0.998046875],
 [-0.233642578125, 0.0919189453125, 0.9971923828125],
 [-0.234375, 0.09130859375, 0.9979248046875],
 [-0.2337646484375, 0.0906982421875, 0.9984130859375],
 [-0.234375, 0.0914306640625, 0.9981689453125],
 [-0.2337646484375, 0.0908203125, 0.9969482421875],
 [-0.234130859375, 0.09228515625, 0.9986572265625],
 [-0.2340087890625, 0.0919189453125, 0.9979248046875],
 [-0.234619140625, 0.092041015625, 0.996337890625],
 [-0.2340087890625, 0.092041015625, 0.998779296875],
 [-0.2342529296875, 0.0908203125, 0.9974365234375],
 [-0.2335205078125, 0.09130859375, 0.9971923828125],
 [-0.233642578125, 0.091552734375, 0.9964599609375],
 [-0.234375, 0.09130859375, 0.9974365234375],
 [-0.2342529296875, 0.0914306640625, 0.997314453125],
 [-0.2337646484375, 0.0919189453125, 0.9976806640625],
 [-0.234130859375, 0.0914306640625, 0.99755859375],
 [-0.2337646484375, 0.092041015625, 0.999267578125],
 [-0.234130859375, 0.091064453125, 0.9986572265625],
 [-0.234130859375, 0.0919189453125, 0.9976806640625],
 [-0.2340087890625, 0.0906982421875, 0.999267578125],
 [-0.233154296875, 0.0906982421875, 0.9990234375],
 [-0.233642578125, 0.091796875, 0.9970703125],
 [-0.2327880859375, 0.0909423828125, 0.9964599609375],
 [-0.233642578125, 0.0921630859375, 0.998291015625],
 [-0.2342529296875, 0.0911865234375, 0.9970703125],
 [-0.23388671875, 0.091552734375, 0.9984130859375],
 [-0.2335205078125, 0.0919189453125, 0.998291015625],
 [-0.2333984375, 0.0911865234375, 0.9981689453125],
 [-0.234130859375, 0.0911865234375, 0.9984130859375],
 [-0.234375, 0.091064453125, 0.998291015625],
 [-0.234130859375, 0.091552734375, 0.9993896484375],
 [-0.2332763671875, 0.092041015625, 0.9986572265625],
 [-0.2335205078125, 0.092041015625, 0.99658203125],
 [-0.233642578125, 0.09130859375, 0.9971923828125],
 [-0.233154296875, 0.091552734375, 0.9981689453125],
 [-0.2332763671875, 0.0911865234375, 0.9986572265625],
 [-0.2342529296875, 0.0906982421875, 0.9964599609375],
 [-0.2342529296875, 0.09130859375, 0.9984130859375],
 [-0.23486328125, 0.091796875, 0.997314453125],
 [-0.233642578125, 0.0909423828125, 0.9974365234375],
 [-0.2332763671875, 0.0921630859375, 0.9989013671875],
 [-0.234375, 0.0921630859375, 0.999267578125],
 [-0.233154296875, 0.0906982421875, 0.998046875],
 [-0.233642578125, 0.091064453125, 0.9981689453125],
 [-0.2340087890625, 0.0914306640625, 0.9969482421875],
 [-0.2335205078125, 0.091796875, 1.0003662109375],
 [-0.2349853515625, 0.091796875, 0.996826171875],
 [-0.233642578125, 0.092041015625, 0.9990234375],
 [-0.234619140625, 0.0916748046875, 0.9986572265625],
 [-0.2337646484375, 0.0908203125, 0.998046875],
 [-0.234130859375, 0.091064453125, 0.9969482421875],
 [-0.2332763671875, 0.09130859375, 0.9998779296875],
 [-0.2344970703125, 0.092529296875, 0.9981689453125],
 [-0.2340087890625, 0.0911865234375, 0.9984130859375],
 [-0.2340087890625, 0.092041015625, 0.9984130859375],
 [-0.2340087890625, 0.0914306640625, 0.997314453125],
 [-0.2327880859375, 0.091796875, 0.9976806640625],
 [-0.2344970703125, 0.0916748046875, 0.9967041015625],
 [-0.2340087890625, 0.0916748046875, 0.996826171875],
 [-0.2332763671875, 0.092041015625, 0.9976806640625],
 [-0.233642578125, 0.0914306640625, 0.99755859375],
 [-0.234375, 0.091796875, 0.9969482421875],
 [-0.2335205078125, 0.0921630859375, 0.996826171875],
 [-0.2333984375, 0.09130859375, 0.998291015625],
 [-0.234375, 0.0906982421875, 0.999755859375],
 [-0.23388671875, 0.0914306640625, 0.9974365234375],
 [-0.2335205078125, 0.0909423828125, 0.9984130859375]])